<a href="https://colab.research.google.com/github/gseetha04/Transfer-learning/blob/main/Transferlearning_experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
##############Fine-tune on SCITE and test on SCITE####################################

In [ ]:
!pip install transformers accelerate seqeval datasets>=1.8.0 torch>=1.3

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
# tokenizer
from transformers import AutoTokenizer, DistilBertTokenizerFast
# sequence tagging model + training-related
from transformers import DistilBertForTokenClassification, Trainer, TrainingArguments
import numpy as np
import pandas as pd
import torch
import json
import sys
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
model_name = "distilbert-base-cased" # "distilbert-base-uncased"
#tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer = DistilBertTokenizerFast.from_pretrained(model_name)
# convenience function for wordpiece tokenization of a list of tokens
tokenize = lambda ds: tokenizer(ds, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)
RANDOM_SEED = 42

(…)cased/resolve/main/tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

(…)ilbert-base-cased/resolve/main/vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

(…)t-base-cased/resolve/main/tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

(…)bert-base-cased/resolve/main/config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

In [ ]:
# special tokens
tokenizer.special_tokens_map

{'unk_token': '[UNK]',
 'sep_token': '[SEP]',
 'pad_token': '[PAD]',
 'cls_token': '[CLS]',
 'mask_token': '[MASK]'}

In [ ]:
data_train = pd.read_excel('/content/train_data.xlsx',sheet_name = "Sheet5")
data_train.head(4)

,sentences,Bio_label
0,"[""A"",""rare"",""and"",""incurable"",""congenital"",""di...","[""C"",""C"",""C"",""C"",""C"",""C"",""O"",""O"",""E"",""O"",""O"",""..."
1,"[""The"",""soft"",""brown"",""and"",""beige"",""tones"",""i...","[""C"",""C"",""C"",""C"",""C"",""C"",""O"",""O"",""O"",""O"",""E"",""..."
2,"[""The"",""JFK"",""baggage"",""system"",""malfunction"",...","[""C"",""C"",""C"",""C"",""C"",""O"",""E"",""O"",""O"",""O"",""O"",""..."
3,"[""Cobalt"",""metal"",""fume"",""and"",""dust"",""cause"",...","[""C"",""C"",""C"",""C"",""C"",""O"",""E"",""O"",""O"",""O"",""O"",""O""]"


In [ ]:
data_test = pd.read_excel('/content/test_data.xlsx',sheet_name = "Sheet3")
data_test.head(4)

,sentences,Bio_label
0,"[""Various"",""hormonal"","","",""bacterial"",""and"",""i...","[""C"",""C"",""C"",""C"",""C"",""C"",""C"",""O"",""E""]"
1,"[""A"",""stereo"",""buss"",""outputs"",""the"",""stereo"",...","[""C"",""C"",""C"",""O"",""E"",""E"",""E"",""E"",""O"",""C"",""C"",""..."
2,"[""The"",""incoming"",""water"",""caused"",""a"",""stain""...","[""C"",""C"",""C"",""O"",""E"",""E"",""O"",""O"",""O"",""O"",""O"",""..."
3,"[""The"",""genreal"",""anesthetic"",""cause"",""unconsc...","[""C"",""C"",""C"",""O"",""E"",""O"",""E"",""O"",""O"",""O"",""O"",""..."


In [ ]:
data_test

,sentences,Bio_label
0,"[""Various"",""hormonal"","","",""bacterial"",""and"",""i...","[""C"",""C"",""C"",""C"",""C"",""C"",""C"",""O"",""E""]"
1,"[""A"",""stereo"",""buss"",""outputs"",""the"",""stereo"",...","[""C"",""C"",""C"",""O"",""E"",""E"",""E"",""E"",""O"",""C"",""C"",""..."
2,"[""The"",""incoming"",""water"",""caused"",""a"",""stain""...","[""C"",""C"",""C"",""O"",""E"",""E"",""O"",""O"",""O"",""O"",""O"",""..."
3,"[""The"",""genreal"",""anesthetic"",""cause"",""unconsc...","[""C"",""C"",""C"",""O"",""E"",""O"",""E"",""O"",""O"",""O"",""O"",""..."
4,"[""The"",""alleged"",""abuse"",""resulted"",""in"",""brui...","[""C"",""C"",""C"",""O"",""O"",""E"",""O"",""E"",""O"",""O"",""O""]"
...,...,...
186,"[""Thus"","","",""evaluating"",""capital"",""punishment...","[""O"",""O"",""O"",""O"",""O"",""O"",""O"",""O"",""O"",""O"",""O"",""..."
187,"[""About"",""30"",""ducks"",""were"",""found"",""dead"",""i...","[""O"",""O"",""O"",""O"",""O"",""O"",""O"",""O"",""O"",""O"",""O"",""..."
188,"[""He"",""created"",""and"",""advocated"",""-"",""flower""...","[""O"",""O"",""O"",""O"",""O"",""O"",""O"",""O"",""O"",""O"",""O"",""..."
189,"[""Method"",""according"",""to"",""claim"",""1"","","",""ch...","[""O"",""O"",""O"",""O"",""O"",""O"",""O"",""O"",""O"",""O"",""O"",""..."


In [ ]:
# load data and rename relevant columns to X (input) and y (output)
dataset = data_train[["sentences", "Bio_label"]].rename(columns={"sentences": "X", "Bio_label": "y"})
# load each row's data as a list of strings
X_raw, y_raw = [json.loads(tokens) for tokens in dataset.X.values], [json.loads(labels) for labels in dataset.y.values]


In [ ]:
# load data and rename relevant columns to X (input) and y (output)
dataset_test = data_test[["sentences", "Bio_label"]].rename(columns={"sentences": "A", "Bio_label": "B"})
# load each row's data as a list of strings
X_raw_test, y_raw_test = [json.loads(tokens_test) for tokens_test in dataset_test.A.values], [json.loads(labels_test) for labels_test in dataset_test.B.values]


In [ ]:
X_raw_test

[['Various',
  'hormonal',
  ',',
  'bacterial',
  'and',
  'inflammatory',
  'disturbances',
  'cause',
  'acne'],
 ['A',
  'stereo',
  'buss',
  'outputs',
  'the',
  'stereo',
  'buss',
  'signal',
  ';',
  'a',
  'record',
  'buss',
  'outputs',
  'the',
  'record',
  'buss',
  'signal'],
 ['The',
  'incoming',
  'water',
  'caused',
  'a',
  'stain',
  'on',
  'the',
  'wall',
  'that',
  ',',
  'through',
  'the',
  'lens',
  'of',
  'pareidolia',
  ',',
  'looked',
  'like',
  'a',
  'bearded',
  'man',
  'with',
  'a',
  'big',
  'afro',
  'hairdo'],
 ['The',
  'genreal',
  'anesthetic',
  'cause',
  'unconsciousness',
  'and',
  'insensibility',
  'to',
  'paid',
  'and',
  'are',
  'used',
  'for',
  'major',
  'surgical',
  'procedures'],
 ['The',
  'alleged',
  'abuse',
  'resulted',
  'in',
  'bruises',
  'and',
  'swelling',
  'of',
  'the',
  'brain'],
 ["Obama's",
  'economic',
  'policies',
  'are',
  'turning',
  'into',
  'a',
  'global',
  'disaster'],
 ['These',
  

In [ ]:
y_raw_test

[['C', 'C', 'C', 'C', 'C', 'C', 'C', 'O', 'E'],
 ['C',
  'C',
  'C',
  'O',
  'E',
  'E',
  'E',
  'E',
  'O',
  'C',
  'C',
  'C',
  'O',
  'E',
  'E',
  'E',
  'E'],
 ['C',
  'C',
  'C',
  'O',
  'E',
  'E',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['C',
  'C',
  'C',
  'O',
  'E',
  'O',
  'E',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['C', 'C', 'C', 'O', 'O', 'E', 'O', 'E', 'O', 'O', 'O'],
 ['C', 'C', 'C', 'O', 'O', 'O', 'E', 'E', 'E'],
 ['C',
  'C',
  'C',
  'O',
  'O',
  'O',
  'E',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['C', 'C', 'C', 'O', 'O', 'O', 'O', 'O', 'E', 'E'],
 ['C',
  'C',
  'C',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'E',
  'E',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['C',
  'C',
  'C',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O

In [ ]:
unique_tags = set(label for labels in y_raw for label in labels)
tag2id = {tag: id for id, tag in enumerate(unique_tags)}
id2tag = {id: tag for tag, id in tag2id.items()}

In [ ]:
id2tag

{0: 'E', 1: 'C', 2: 'O', 3: 'CE'}

In [ ]:
# Our data is split into sentences


# FIXME: we also need test data!
X_train_raw, X_dev_raw, y_train_raw, y_dev_raw = train_test_split(X_raw, y_raw, test_size=.2, random_state=RANDOM_SEED)

In [ ]:
# ensure our data is well-formed:
for ds in [X_train_raw, X_train_raw]:
  for row in ds:
    #print(row)
    assert all(len(tok) > 0 for tok in row)

In [ ]:
# ensure our data is well-formed:
for ds in [X_dev_raw, X_dev_raw]:
  for row in ds:
    #print(row)
    assert all(len(tok) > 0 for tok in row)

In [ ]:
# ensure our data is well-formed:
for ds in [X_raw_test, X_raw_test]:
  for row in ds:
    #print(row)
    assert all(len(tok) > 0 for tok in row)

In [ ]:
def encode_tags(tags, encodings, tag2id):
    """
    See https://huggingface.co/transformers/custom_datasets.html#tok-ner

    >If the tokenizer splits a token into multiple sub-tokens, then we will end up with a mismatch between our tokens and our labels.
    >One way to handle this is to only train on the tag labels for the first subtoken of a split token.
    >We can do this in 🤗 Transformers by setting the labels we wish to ignore to -100.
    >In the example above, if the label for @HuggingFace is 3 (indexing `B-corporation`),
    >we would set the labels of ['@', 'hugging', '##face'] to [3, -100, -100].
    """
    labels = [[tag2id[tag] for tag in doc] for doc in tags]
    encoded_labels = []
    for doc_labels, doc_offset in zip(labels, encodings.offset_mapping):
        # create an empty array of -100
        doc_enc_labels = np.ones(len(doc_offset),dtype=int) * -100
        arr_offset = np.array(doc_offset)

        # set labels whose first offset position is 0 and the second is not 0
        doc_enc_labels[(arr_offset[:,0] == 0) & (arr_offset[:,1] != 0)] = doc_labels
        encoded_labels.append(doc_enc_labels.tolist())


    return encoded_labels

In [ ]:
# encoded X
X_train = tokenize(X_train_raw)
X_dev = tokenize(X_dev_raw)
X_test= tokenize(X_raw_test)

y_train = encode_tags(tags=y_train_raw, encodings=X_train, tag2id=tag2id)
y_dev = encode_tags(tags=y_dev_raw, encodings=X_dev, tag2id=tag2id)
y_test = encode_tags(tags=y_raw_test, encodings=X_test, tag2id=tag2id)


for X in [X_train, X_dev, X_test]:
  X.pop("offset_mapping")

In [ ]:
class CustomDataset(torch.utils.data.Dataset):
    """
    Pytorch dataset for easy batching
    """
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = CustomDataset(X_train, y_train)
val_dataset = CustomDataset(X_dev, y_dev)
test_dataset = CustomDataset(X_test,y_test)

NameError: ignored

In [ ]:
training_args = TrainingArguments(
    output_dir='/content/results/',   # output directory
    num_train_epochs=3,                   # total number of training epochs
    per_device_train_batch_size=16,       # batch size per device during training
    per_device_eval_batch_size=64,        # batch size for evaluation
    warmup_steps=500,                     # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                    # strength of weight decay
    logging_dir='/content/results',     # directory for storing logs
    logging_steps=10,
    evaluation_strategy = "epoch",
    #report_to = "wandb"
)

model = DistilBertForTokenClassification.from_pretrained(model_name, num_labels=len(unique_tags))

trainer = Trainer(
    model=model,                          # the instantiated 🤗 Transformers model to be trained
    args=training_args,                   # training arguments, defined above
    train_dataset=train_dataset,          # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    #compute_metrics = compute_metrics
)

trainer.train()
pred = trainer.predict(test_dataset)
preds = pred.predictions.argmax(-1)
#indices = val_index
#final_pred = pd.Series(preds).tolist()

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.976200,0.887334
2,0.659700,0.549919
3,0.229200,0.198399


In [ ]:
flat_list = [item for sublist in preds for item in sublist]

In [ ]:
np.shape(flat_list)

(13561,)

In [ ]:
flat_list1 = [item1 for sublist1 in y_test for item1 in sublist1]

In [ ]:
np.shape(flat_list1)

(13561,)

In [ ]:
lis_preds_frame = pd.DataFrame(flat_list)
lis_test_labels_frame = pd.DataFrame(flat_list1)
frame_combine_test = pd.concat([lis_preds_frame,lis_test_labels_frame],axis=1)
frame_combine_test.columns = ['preds','gold_data']

In [ ]:
frame_combine_test

,preds,gold_data
0,0,-100
1,3,3
2,3,3
3,0,-100
4,0,-100
...,...,...
13556,0,-100
13557,0,-100
13558,0,-100
13559,0,-100


In [ ]:
frame_combine_test_new = frame_combine_test[frame_combine_test.gold_data != -100]
frame_combine_test_new

,preds,gold_data
1,3,3
2,3,3
5,0,3
6,3,3
7,0,3
...,...,...
13519,0,3
13520,3,3
13521,0,0
13522,0,0


In [ ]:
print(id2tag)

{0: 'O', 1: 'E', 2: 'CE', 3: 'C'}


In [ ]:
targetname = ['O','E', 'CE','C']

In [ ]:
########SCITE as train and test#######################

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()

              precision    recall  f1-score   support

           O       0.96      0.96      0.96      2663
           E       0.87      0.85      0.86       468
          CE       1.00      0.00      0.00        25
           C       0.85      0.88      0.86       465

    accuracy                           0.93      3621
   macro avg       0.92      0.67      0.67      3621
weighted avg       0.93      0.93      0.93      3621



In [ ]:
training_args = TrainingArguments(
    output_dir='/content/results/',   # output directory
    num_train_epochs=3,                   # total number of training epochs
    per_device_train_batch_size=16,       # batch size per device during training
    per_device_eval_batch_size=64,        # batch size for evaluation
    warmup_steps=500,                     # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                    # strength of weight decay
    logging_dir='/content/results',     # directory for storing logs
    logging_steps=10,
    evaluation_strategy = "epoch",
    #report_to = "wandb"
)

model = DistilBertForTokenClassification.from_pretrained(model_name, num_labels=len(unique_tags))

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from sklearn.metrics import classification_report

In [ ]:

for i in range(10):
  trainer = Trainer(
    model=model,                          # the instantiated 🤗 Transformers model to be trained
    args=training_args,                   # training arguments, defined above
    train_dataset=train_dataset,          # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    #compute_metrics = compute_metrics
  )

  trainer.train()
  pred = trainer.predict(test_dataset)
  preds = pred.predictions.argmax(-1)
  flat_list = [item for sublist in preds for item in sublist]
  flat_list1 = [item1 for sublist1 in y_test for item1 in sublist1]

  lis_preds_frame = pd.DataFrame(flat_list)
  lis_test_labels_frame = pd.DataFrame(flat_list1)
  frame_combine_test = pd.concat([lis_preds_frame,lis_test_labels_frame],axis=1)
  frame_combine_test.columns = ['preds','gold_data']

  frame_combine_test_new = frame_combine_test[frame_combine_test.gold_data != -100]
  print(frame_combine_test_new)

  print(id2tag)
  targetname = ['E','C', 'O','CE']


  print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()

Epoch,Training Loss,Validation Loss
1,1.002200,0.884354
2,0.620400,0.492095
3,0.219700,0.202629


       preds  gold_data
1          1          1
2          1          1
5          2          1
6          1          1
7          2          1
...      ...        ...
13519      1          1
13520      1          1
13521      2          2
13522      2          2
13523      2          2

[3621 rows x 2 columns]
{0: 'E', 1: 'C', 2: 'O', 3: 'CE'}
              precision    recall  f1-score   support

           E       0.84      0.91      0.87       468
           C       0.83      0.89      0.86       465
           O       0.97      0.95      0.96      2663
          CE       1.00      0.00      0.00        25

    accuracy                           0.93      3621
   macro avg       0.91      0.69      0.67      3621
weighted avg       0.94      0.93      0.93      3621



Epoch,Training Loss,Validation Loss
1,0.155600,0.188293
2,0.144700,0.165999
3,0.092800,0.181264


       preds  gold_data
1          1          1
2          1          1
5          1          1
6          1          1
7          1          1
...      ...        ...
13519      1          1
13520      1          1
13521      2          2
13522      2          2
13523      2          2

[3621 rows x 2 columns]
{0: 'E', 1: 'C', 2: 'O', 3: 'CE'}
              precision    recall  f1-score   support

           E       0.90      0.89      0.89       468
           C       0.83      0.93      0.88       465
           O       0.97      0.96      0.97      2663
          CE       1.00      0.00      0.00        25

    accuracy                           0.94      3621
   macro avg       0.93      0.70      0.69      3621
weighted avg       0.95      0.94      0.94      3621



Epoch,Training Loss,Validation Loss
1,0.072700,0.165947
2,0.059400,0.165820
3,0.043300,0.186100


...

       preds  gold_data
1          1          1
2          1          1
5          1          1
6          1          1
7          1          1
...      ...        ...
13519      1          1
13520      1          1
13521      2          2
13522      2          2
13523      2          2

[3621 rows x 2 columns]
{0: 'E', 1: 'C', 2: 'O', 3: 'CE'}
              precision    recall  f1-score   support

           E       0.90      0.88      0.89       468
           C       0.85      0.92      0.89       465
           O       0.97      0.96      0.97      2663
          CE       0.44      0.28      0.34        25

    accuracy                           0.94      3621
   macro avg       0.79      0.76      0.77      3621
weighted avg       0.94      0.94      0.94      3621



Epoch,Training Loss,Validation Loss
1,0.003000,0.256585
2,0.000300,0.262905
3,0.000300,0.282405


       preds  gold_data
1          1          1
2          1          1
5          1          1
6          1          1
7          1          1
...      ...        ...
13519      1          1
13520      1          1
13521      2          2
13522      2          2
13523      2          2

[3621 rows x 2 columns]
{0: 'E', 1: 'C', 2: 'O', 3: 'CE'}
              precision    recall  f1-score   support

           E       0.94      0.85      0.89       468
           C       0.88      0.90      0.89       465
           O       0.96      0.98      0.97      2663
          CE       0.36      0.20      0.26        25

    accuracy                           0.94      3621
   macro avg       0.78      0.73      0.75      3621
weighted avg       0.94      0.94      0.94      3621



Epoch,Training Loss,Validation Loss
1,0.000200,0.276682
2,0.000200,0.283680
3,0.000100,0.299036


       preds  gold_data
1          1          1
2          1          1
5          1          1
6          1          1
7          1          1
...      ...        ...
13519      1          1
13520      1          1
13521      2          2
13522      2          2
13523      2          2

[3621 rows x 2 columns]
{0: 'E', 1: 'C', 2: 'O', 3: 'CE'}
              precision    recall  f1-score   support

           E       0.94      0.85      0.89       468
           C       0.88      0.90      0.89       465
           O       0.96      0.98      0.97      2663
          CE       0.50      0.20      0.29        25

    accuracy                           0.95      3621
   macro avg       0.82      0.73      0.76      3621
weighted avg       0.94      0.95      0.94      3621



In [ ]:
#############SCITE as train, Fincausal as test#########################

In [ ]:
data_test = pd.read_json('/content/Fincausal-test.json', lines=True)
data_test.head(4)

,words,ner,id,postfix
0,"[During, 2020, ,, the, Company, also, issued, ...","[E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, ...",3,"[ , , , , , , , , , , , , , , , , ..."
1,"[The, significant, increases, in, earnings, as...","[E, E, E, E, E, E, E, E, O, O, O, O, C, C, C, ...",4,"[ , , , , , , , , , , , , , , , ..."
2,"[The, provision, for, credit, losses, was, $, ...","[E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, ...",7,"[ , , , , , , , , , , , , , , , , ..."
3,"[Higher, net, charge-offs, also, contributed, ...","[E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, ...",8,"[ , , , , , , , , , , , , , , , ..."


In [ ]:
# for tokens_test in data_test['words']:
#   s1 = json.dumps(tokens_test)
#   X_raw_test = json.loads(s1)

In [ ]:
# for tokens_test in data_test['ner']:
#   s2 = json.dumps(tokens_test)
#   y_raw_test = json.loads(s2)

In [ ]:
# load data and rename relevant columns to X (input) and y (output)
dataset_test = data_test[["words", "ner"]].rename(columns={"words": "A", "ner": "B"})
# load each row's data as a list of strings
s1, s2 = [json.dumps(tokens_test) for tokens_test in dataset_test.A.values], [json.dumps(labels_test) for labels_test in dataset_test.B.values]

X_raw_test, y_raw_test = [json.loads(token_test) for token_test in s1], [json.loads(label_test) for label_test in s2]


In [ ]:
X_test= tokenize(X_raw_test)

In [ ]:
y_test = encode_tags(tags=y_raw_test, encodings=X_test, tag2id=tag2id)

In [ ]:
test_dataset = CustomDataset(X_test,y_test)

In [ ]:
training_args = TrainingArguments(
    output_dir='/content/results/',   # output directory
    num_train_epochs=3,                   # total number of training epochs
    per_device_train_batch_size=16,       # batch size per device during training
    per_device_eval_batch_size=64,        # batch size for evaluation
    warmup_steps=500,                     # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                    # strength of weight decay
    logging_dir='/content/results',     # directory for storing logs
    logging_steps=10,
    evaluation_strategy = "epoch",
    #report_to = "wandb"
)

model = DistilBertForTokenClassification.from_pretrained(model_name, num_labels=len(unique_tags))

trainer = Trainer(
    model=model,                          # the instantiated 🤗 Transformers model to be trained
    args=training_args,                   # training arguments, defined above
    train_dataset=train_dataset,          # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    #compute_metrics = compute_metrics
)

trainer.train()
pred = trainer.predict(test_dataset)
preds = pred.predictions.argmax(-1)
#indices = val_index
#final_pred = pd.Series(preds).tolist()

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForTokenClassification: ['vocab_projector.bias', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this 

Epoch,Training Loss,Validation Loss
1,1.196000,1.011827
2,0.668200,0.562584
3,0.295900,0.233892


In [ ]:
flat_list = [item for sublist in preds for item in sublist]

In [ ]:
np.shape(flat_list)

(55650,)

In [ ]:
flat_list1 = [item1 for sublist1 in y_test for item1 in sublist1]

In [ ]:
np.shape(flat_list1)

(55650,)

In [ ]:
lis_preds_frame = pd.DataFrame(flat_list)
lis_test_labels_frame = pd.DataFrame(flat_list1)
frame_combine_test = pd.concat([lis_preds_frame,lis_test_labels_frame],axis=1)
frame_combine_test.columns = ['preds','gold_data']

In [ ]:
frame_combine_test

,preds,gold_data
0,0,-100
1,0,1
2,0,1
3,0,1
4,0,1
...,...,...
55645,3,-100
55646,0,-100
55647,0,-100
55648,0,-100


In [ ]:
frame_combine_test_new = frame_combine_test[frame_combine_test.gold_data != -100]
frame_combine_test_new

,preds,gold_data
1,0,1
2,0,1
3,0,1
4,0,1
5,0,1
...,...,...
55505,0,3
55506,0,3
55507,0,3
55508,0,3


In [ ]:
print(id2tag)

{0: 'O', 1: 'E', 2: 'CE', 3: 'C'}


In [ ]:
targetname = ['O','E','C']

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()

              precision    recall  f1-score   support

           O       0.05      1.00      0.10       640
           E       0.68      0.02      0.04      5079
           C       0.92      0.06      0.11      6590

    accuracy                           0.09     12309
   macro avg       0.55      0.36      0.09     12309
weighted avg       0.78      0.09      0.09     12309



In [ ]:
######10 runs

In [ ]:

for i in range(10):
  trainer = Trainer(
    model=model,                          # the instantiated 🤗 Transformers model to be trained
    args=training_args,                   # training arguments, defined above
    train_dataset=train_dataset,          # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    #compute_metrics = compute_metrics
  )

  trainer.train()
  pred = trainer.predict(test_dataset)
  preds = pred.predictions.argmax(-1)
  flat_list = [item for sublist in preds for item in sublist]
  flat_list1 = [item1 for sublist1 in y_test for item1 in sublist1]

  lis_preds_frame = pd.DataFrame(flat_list)
  lis_test_labels_frame = pd.DataFrame(flat_list1)
  frame_combine_test = pd.concat([lis_preds_frame,lis_test_labels_frame],axis=1)
  frame_combine_test.columns = ['preds','gold_data']

  frame_combine_test_new = frame_combine_test[frame_combine_test.gold_data != -100]
  print(frame_combine_test_new)

  print(id2tag)
  targetname = ['E','C', 'O','CE']


  print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()

Epoch,Training Loss,Validation Loss
1,0.000100,0.299711
2,0.000300,0.320347
3,0.006700,0.320368


       preds  gold_data
1          2          0
2          2          0
3          2          0
4          2          0
5          2          0
...      ...        ...
55505      2          1
55506      2          1
55507      2          1
55508      2          1
55509      2          1

[12309 rows x 2 columns]
{0: 'E', 1: 'C', 2: 'O', 3: 'CE'}
              precision    recall  f1-score   support

           E       0.58      0.07      0.12      5079
           C       0.91      0.13      0.22      6590
           O       0.06      1.00      0.11       640
          CE       0.00      1.00      0.00         0

    accuracy                           0.15     12309
   macro avg       0.39      0.55      0.11     12309
weighted avg       0.73      0.15      0.18     12309



Epoch,Training Loss,Validation Loss
1,0.000100,0.309665
2,0.000100,0.309402
3,0.004200,0.334011


       preds  gold_data
1          2          0
2          2          0
3          2          0
4          2          0
5          2          0
...      ...        ...
55505      2          1
55506      2          1
55507      2          1
55508      2          1
55509      2          1

[12309 rows x 2 columns]
{0: 'E', 1: 'C', 2: 'O', 3: 'CE'}
              precision    recall  f1-score   support

           E       0.54      0.08      0.14      5079
           C       0.92      0.15      0.26      6590
           O       0.06      0.99      0.12       640
          CE       0.00      1.00      0.00         0

    accuracy                           0.17     12309
   macro avg       0.38      0.56      0.13     12309
weighted avg       0.72      0.17      0.21     12309



Epoch,Training Loss,Validation Loss
1,0.000100,0.313360
2,0.000100,0.320609
3,0.000100,0.329315


...

       preds  gold_data
1          2          0
2          2          0
3          2          0
4          2          0
5          2          0
...      ...        ...
55505      2          1
55506      2          1
55507      2          1
55508      2          1
55509      2          1

[12309 rows x 2 columns]
{0: 'E', 1: 'C', 2: 'O', 3: 'CE'}
              precision    recall  f1-score   support

           E       0.55      0.06      0.11      5079
           C       0.92      0.14      0.25      6590
           O       0.06      0.99      0.11       640
          CE       0.00      1.00      0.00         0

    accuracy                           0.15     12309
   macro avg       0.38      0.55      0.12     12309
weighted avg       0.72      0.15      0.18     12309



Epoch,Training Loss,Validation Loss
1,0.000000,0.460475
2,0.001300,0.493178
3,0.001300,0.471903


       preds  gold_data
1          2          0
2          2          0
3          2          0
4          2          0
5          2          0
...      ...        ...
55505      2          1
55506      2          1
55507      2          1
55508      2          1
55509      2          1

[12309 rows x 2 columns]
{0: 'E', 1: 'C', 2: 'O', 3: 'CE'}
              precision    recall  f1-score   support

           E       0.60      0.07      0.13      5079
           C       0.91      0.11      0.20      6590
           O       0.06      0.99      0.11       640
          CE       0.00      1.00      0.00         0

    accuracy                           0.14     12309
   macro avg       0.39      0.54      0.11     12309
weighted avg       0.73      0.14      0.16     12309



Epoch,Training Loss,Validation Loss
1,0.000000,0.465132
2,0.000000,0.485321
3,0.002200,0.555319


       preds  gold_data
1          2          0
2          2          0
3          2          0
4          2          0
5          2          0
...      ...        ...
55505      2          1
55506      2          1
55507      2          1
55508      2          1
55509      2          1

[12309 rows x 2 columns]
{0: 'E', 1: 'C', 2: 'O', 3: 'CE'}
              precision    recall  f1-score   support

           E       0.54      0.13      0.21      5079
           C       0.92      0.14      0.25      6590
           O       0.06      0.99      0.12       640
          CE       0.00      1.00      0.00         0

    accuracy                           0.18     12309
   macro avg       0.38      0.56      0.14     12309
weighted avg       0.72      0.18      0.22     12309



In [ ]:
######################SCITE as train and Organizational data as test###########################

In [ ]:
# mount Google Drive
from google.colab import drive
import os

drive.mount("/content/drive", force_remount=True)

# data location
#parent_dir = "/content/drive/MyDrive"
parent_dir = "/content/drive/MyDrive/"
mk_data_path = lambda filename: os.path.join(parent_dir, filename)
#data_path = f"{parent_dir}/BERT_data_final.xlsx"
data_path = mk_data_path("BERT_data_final1.xlsx")

# once finished, flush changes to files to make them visible (assuming we write to Google Drive)
# drive.flush_and_unmount()

Mounted at /content/drive


In [ ]:
data = pd.read_excel(data_path)
data.head(4)

,Causal-relation text,BIO Code,"{""words"":Causal-relation text,""ner"":BIO Code}",BIO_Code_changed,BIO Code-changed-NER
0,"[""When "",""a "",""policyholder "",""or "",""insured ""...","[""O"",""O"",""B-C"",""I-C"",""I-C"",""I-C"",""I-C"",""I-C"",""...","{""words"":[""When "",""a "",""policyholder "",""or "",""...","[""O"",""O"",""C"",""C"",""C"",""C"",""C"",""C"",""C"",""O"",""O"",""...","[""O"",""O"",""B-ENTITY"",""B-ENTITY"",""B-ENTITY"",""B-E..."
1,"[""During "",""2020 "",""in "",""response "",""to "",""th...","[""O"",""O"",""O"",""O"",""O"",""B-C"",""I-C"",""I-C"",""I-C"",""...","{""words"":[""During "",""2020 "",""in "",""response "",...","[""O"",""O"",""O"",""O"",""O"",""C"",""C"",""C"",""C"",""C"",""C"",""...","[""O"",""O"",""O"",""O"",""O"",""B-ENTITY"",""B-ENTITY"",""B-..."
2,"[""Prolonged "",""periods "",""of "",""low "",""interes...","[""B-C"",""I-C"",""I-C"",""I-C"",""I-C"",""I-C"",""O"",""B-CT...","{""words"":[""Prolonged "",""periods "",""of "",""low ""...","[""C"",""C"",""C"",""C"",""C"",""C"",""O"",""CT"",""E"",""E"",""O"",...","[""B-ENTITY"",""B-ENTITY"",""B-ENTITY"",""B-ENTITY"",""..."
3,"[""Conversely "",""a "",""rise "",""in "",""interest "",...","[""O"",""O"",""B-C"",""I-C"",""I-C"",""I-C"",""O"",""B-CT"",""O...","{""words"":[""Conversely "",""a "",""rise "",""in "",""in...","[""O"",""O"",""C"",""C"",""C"",""C"",""O"",""CT"",""O"",""O"",""O"",...","[""O"",""O"",""B-ENTITY"",""B-ENTITY"",""B-ENTITY"",""B-E..."


In [ ]:
# load data and rename relevant columns to X (input) and y (output)
dataset = data[["Causal-relation text", "BIO_Code_changed"]].rename(columns={"Causal-relation text": "X", "BIO_Code_changed": "y"})
# load each row's data as a list of strings
X_raw, y_raw = [json.loads(tokens) for tokens in dataset.X.values], [json.loads(labels) for labels in dataset.y.values]

In [ ]:
# Our data is split into sentences


# FIXME: we also need test data!
X_train_raw, X_dev_raw, y_train_raw, y_dev_raw = train_test_split(X_raw, y_raw, test_size=.2, random_state=RANDOM_SEED)

In [ ]:
X_test= tokenize(X_dev_raw)

In [ ]:
for i, x in enumerate(y_dev_raw):
    for j, a in enumerate(x):
        if 'CT' in a:
            y_dev_raw[i][j] = a.replace('CT', 'CE')

In [ ]:
y_dev_raw

[['C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'CE',
  'CE',
  'CE',
  'O',
  'O',
  'E',
  'E',
  'E',
  'E',
  'E',
  'E',
  'E',
  'E',
  'E',
  'E'],
 ['CE',
  'CE',
  'CE',
  'E',
  'E',
  'E',
  'E',
  'E',
  'E',
  'E',
  'E',
  'E',
  'E',
  'E',
  'E',
  'E',
  'E',
  'E',
  'E',
  'E',
  'E',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'C',
  'C',
  'C',
  'C',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'CE',
  'CE',
  'CE',
  'E',
  'E',
  'E',
  'E',
  'E',
  'E',
  'E',
  'E',
  'E',
  'E',
  'O',
  'E',
  'E',
  'E'],
 ['O',
  'O',
  'E',
  'E',
  'E',
  'E',
  'CE',
  'CE',
  'O',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C'],
 ['O',
  'O'

In [ ]:
y_test = encode_tags(tags=y_dev_raw, encodings=X_test, tag2id=tag2id)

In [ ]:
test_dataset = CustomDataset(X_test,y_test)

In [ ]:
training_args = TrainingArguments(
    output_dir='/content/results/',   # output directory
    num_train_epochs=3,                   # total number of training epochs
    per_device_train_batch_size=16,       # batch size per device during training
    per_device_eval_batch_size=64,        # batch size for evaluation
    warmup_steps=500,                     # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                    # strength of weight decay
    logging_dir='/content/results',     # directory for storing logs
    logging_steps=10,
    evaluation_strategy = "epoch",
    #report_to = "wandb"
)

model = DistilBertForTokenClassification.from_pretrained(model_name, num_labels=len(unique_tags))

trainer = Trainer(
    model=model,                          # the instantiated 🤗 Transformers model to be trained
    args=training_args,                   # training arguments, defined above
    train_dataset=train_dataset,          # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    #compute_metrics = compute_metrics
)

trainer.train()
pred = trainer.predict(test_dataset)
preds = pred.predictions.argmax(-1)
#indices = val_index
#final_pred = pd.Series(preds).tolist()

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForTokenClassification: ['vocab_projector.bias', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this 

Epoch,Training Loss,Validation Loss
1,1.196000,1.011827
2,0.668200,0.562584
3,0.295900,0.233892


In [ ]:
flat_list = [item for sublist in preds for item in sublist]

In [ ]:
np.shape(flat_list)

(52746,)

In [ ]:
flat_list1 = [item1 for sublist1 in y_test for item1 in sublist1]

In [ ]:
np.shape(flat_list1)

(52746,)

In [ ]:
lis_preds_frame = pd.DataFrame(flat_list)
lis_test_labels_frame = pd.DataFrame(flat_list1)
frame_combine_test = pd.concat([lis_preds_frame,lis_test_labels_frame],axis=1)
frame_combine_test.columns = ['preds','gold_data']

In [ ]:
frame_combine_test_new = frame_combine_test[frame_combine_test.gold_data != -100]
frame_combine_test_new

,preds,gold_data
1,0,3
2,0,3
3,0,3
4,0,3
5,0,3
...,...,...
52655,0,1
52656,0,1
52657,0,1
52658,0,1


In [ ]:
print(id2tag)

{0: 'O', 1: 'E', 2: 'CE', 3: 'C'}


In [ ]:
targetname = ['O','E','CE','C']

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()

              precision    recall  f1-score   support

           O       0.15      0.95      0.26      2107
           E       0.66      0.06      0.10      5662
          CE       1.00      0.00      0.00       871
           C       0.70      0.07      0.12      5694

    accuracy                           0.19     14334
   macro avg       0.63      0.27      0.12     14334
weighted avg       0.62      0.19      0.13     14334



In [ ]:
########10 runs##########################################

In [ ]:

for i in range(10):
  trainer = Trainer(
    model=model,                          # the instantiated 🤗 Transformers model to be trained
    args=training_args,                   # training arguments, defined above
    train_dataset=train_dataset,          # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    #compute_metrics = compute_metrics
  )

  trainer.train()
  pred = trainer.predict(test_dataset)
  preds = pred.predictions.argmax(-1)
  flat_list = [item for sublist in preds for item in sublist]
  flat_list1 = [item1 for sublist1 in y_test for item1 in sublist1]

  lis_preds_frame = pd.DataFrame(flat_list)
  lis_test_labels_frame = pd.DataFrame(flat_list1)
  frame_combine_test = pd.concat([lis_preds_frame,lis_test_labels_frame],axis=1)
  frame_combine_test.columns = ['preds','gold_data']

  frame_combine_test_new = frame_combine_test[frame_combine_test.gold_data != -100]
  print(frame_combine_test_new)

  print(id2tag)
  targetname = ['E','C', 'O','CE']


  print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()

Epoch,Training Loss,Validation Loss
1,0.000000,0.451975
2,0.000000,0.456758
3,0.000200,0.447459


       preds  gold_data
1          2          1
2          2          1
3          2          1
4          2          1
5          2          1
...      ...        ...
52655      2          0
52656      2          0
52657      2          0
52658      2          0
52659      2          0

[14334 rows x 2 columns]
{0: 'E', 1: 'C', 2: 'O', 3: 'CE'}
              precision    recall  f1-score   support

           E       0.70      0.08      0.14      5662
           C       0.77      0.11      0.19      5694
           O       0.15      0.92      0.26      2107
          CE       0.00      0.00      0.00       871

    accuracy                           0.21     14334
   macro avg       0.40      0.28      0.15     14334
weighted avg       0.60      0.21      0.17     14334



Epoch,Training Loss,Validation Loss
1,0.000000,0.448169
2,0.000000,0.450710
3,0.000000,0.462252


       preds  gold_data
1          2          1
2          2          1
3          2          1
4          2          1
5          2          1
...      ...        ...
52655      2          0
52656      2          0
52657      2          0
52658      2          0
52659      2          0

[14334 rows x 2 columns]
{0: 'E', 1: 'C', 2: 'O', 3: 'CE'}
              precision    recall  f1-score   support

           E       0.69      0.10      0.18      5662
           C       0.75      0.13      0.22      5694
           O       0.15      0.90      0.26      2107
          CE       0.00      0.00      0.00       871

    accuracy                           0.22     14334
   macro avg       0.40      0.28      0.16     14334
weighted avg       0.60      0.22      0.20     14334



Epoch,Training Loss,Validation Loss
1,0.000000,0.443478
2,0.000000,0.448520
3,0.000000,0.452299


...

       preds  gold_data
1          2          1
2          2          1
3          2          1
4          2          1
5          2          1
...      ...        ...
52655      2          0
52656      2          0
52657      2          0
52658      2          0
52659      2          0

[14334 rows x 2 columns]
{0: 'E', 1: 'C', 2: 'O', 3: 'CE'}
              precision    recall  f1-score   support

           E       0.68      0.11      0.18      5662
           C       0.77      0.13      0.22      5694
           O       0.15      0.91      0.26      2107
          CE       0.00      0.00      0.00       871

    accuracy                           0.23     14334
   macro avg       0.40      0.28      0.17     14334
weighted avg       0.60      0.23      0.20     14334



Epoch,Training Loss,Validation Loss
1,0.000100,0.436677
2,0.000000,0.440171
3,0.000000,0.444582


       preds  gold_data
1          2          1
2          2          1
3          2          1
4          2          1
5          2          1
...      ...        ...
52655      2          0
52656      2          0
52657      2          0
52658      2          0
52659      2          0

[14334 rows x 2 columns]
{0: 'E', 1: 'C', 2: 'O', 3: 'CE'}
              precision    recall  f1-score   support

           E       0.68      0.11      0.18      5662
           C       0.77      0.12      0.21      5694
           O       0.15      0.91      0.26      2107
          CE       0.00      0.00      0.00       871

    accuracy                           0.22     14334
   macro avg       0.40      0.28      0.16     14334
weighted avg       0.60      0.22      0.19     14334



Epoch,Training Loss,Validation Loss
1,0.000100,0.456783
2,0.000000,0.452798
3,0.000000,0.460042


       preds  gold_data
1          2          1
2          2          1
3          2          1
4          2          1
5          2          1
...      ...        ...
52655      2          0
52656      2          0
52657      2          0
52658      2          0
52659      2          0

[14334 rows x 2 columns]
{0: 'E', 1: 'C', 2: 'O', 3: 'CE'}
              precision    recall  f1-score   support

           E       0.68      0.10      0.17      5662
           C       0.77      0.12      0.21      5694
           O       0.15      0.91      0.26      2107
          CE       0.00      0.00      0.00       871

    accuracy                           0.22     14334
   macro avg       0.40      0.28      0.16     14334
weighted avg       0.60      0.22      0.19     14334



In [ ]:
################ FinCausal as train and FinCausal as test#########################################

In [ ]:
import pandas as pd
data_train = pd.read_json('/content/fincausal-extra-all-train.json', lines=True)
data_train.head(4)

,words,ner,id,postfix
0,"[It, found, that, total, U.S., health, care, s...","[E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, ...",0001.00026,"[ , , , , , , , , , , , , , , , ..."
1,"[Transat, loss, more, than, doubles, as, it, w...","[E, E, E, E, E, O, C, C, C, C, C, C, C]",0003.00001,"[ , , , , , , , , , , , ]"
2,"[MONTREAL, -, Transat, AT, Inc., 's, third-qua...","[O, O, E, E, E, E, E, E, E, E, E, E, E, E, E, ...",0003.00002,"[ , , , , , , , , , , , , , , ,..."
3,"[The, Montreal-based, company, lost, $, 11.0, ...","[E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, ...",0003.00003,"[ , , , , , , , , , , , , , , , ,..."


In [ ]:
import json
# load data and rename relevant columns to X (input) and y (output)
dataset_train = data_train[["words", "ner"]].rename(columns={"words": "A", "ner": "B"})
# load each row's data as a list of strings
s1, s2 = [json.dumps(tokens_test) for tokens_test in dataset_train.A.values], [json.dumps(labels_test) for labels_test in dataset_train.B.values]

X_raw, y_raw = [json.loads(token_test) for token_test in s1], [json.loads(label_test) for label_test in s2]

In [ ]:
data_test = pd.read_json('/content/Fincausal-test.json', lines=True)
data_test.head(4)

,words,ner,id,postfix
0,"[During, 2020, ,, the, Company, also, issued, ...","[E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, ...",3,"[ , , , , , , , , , , , , , , , , ..."
1,"[The, significant, increases, in, earnings, as...","[E, E, E, E, E, E, E, E, O, O, O, O, C, C, C, ...",4,"[ , , , , , , , , , , , , , , , ..."
2,"[The, provision, for, credit, losses, was, $, ...","[E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, ...",7,"[ , , , , , , , , , , , , , , , , ..."
3,"[Higher, net, charge-offs, also, contributed, ...","[E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, ...",8,"[ , , , , , , , , , , , , , , , ..."


In [ ]:
# load data and rename relevant columns to X (input) and y (output)
dataset_test = data_test[["words", "ner"]].rename(columns={"words": "A", "ner": "B"})
# load each row's data as a list of strings
s1, s2 = [json.dumps(tokens_test) for tokens_test in dataset_test.A.values], [json.dumps(labels_test) for labels_test in dataset_test.B.values]

X_raw_test, y_raw_test = [json.loads(token_test) for token_test in s1], [json.loads(label_test) for label_test in s2]

In [ ]:
unique_tags = set(label for labels in y_raw for label in labels)
tag2id = {tag: id for id, tag in enumerate(unique_tags)}
id2tag = {id: tag for tag, id in tag2id.items()}

In [ ]:
id2tag

{0: 'E', 1: 'C', 2: 'O'}

In [ ]:
# Our data is split into sentences


# FIXME: we also need test data!
X_train_raw, X_dev_raw, y_train_raw, y_dev_raw = train_test_split(X_raw, y_raw, test_size=.2, random_state=RANDOM_SEED)

In [ ]:
# ensure our data is well-formed:
for ds in [X_train_raw, X_train_raw]:
  for row in ds:
    #print(row)
    assert all(len(tok) > 0 for tok in row)

In [ ]:
# ensure our data is well-formed:
for ds in [X_dev_raw, X_dev_raw]:
  for row in ds:
    #print(row)
    assert all(len(tok) > 0 for tok in row)

In [ ]:
# ensure our data is well-formed:
for ds in [X_raw_test, X_raw_test]:
  for row in ds:
    #print(row)
    assert all(len(tok) > 0 for tok in row)


In [ ]:
def encode_tags(tags, encodings, tag2id):
    """
    See https://huggingface.co/transformers/custom_datasets.html#tok-ner

    >If the tokenizer splits a token into multiple sub-tokens, then we will end up with a mismatch between our tokens and our labels.
    >One way to handle this is to only train on the tag labels for the first subtoken of a split token.
    >We can do this in 🤗 Transformers by setting the labels we wish to ignore to -100.
    >In the example above, if the label for @HuggingFace is 3 (indexing `B-corporation`),
    >we would set the labels of ['@', 'hugging', '##face'] to [3, -100, -100].
    """
    labels = [[tag2id[tag] for tag in doc] for doc in tags]
    encoded_labels = []
    for doc_labels, doc_offset in zip(labels, encodings.offset_mapping):
        # create an empty array of -100
        doc_enc_labels = np.ones(len(doc_offset),dtype=int) * -100
        arr_offset = np.array(doc_offset)

        # set labels whose first offset position is 0 and the second is not 0
        doc_enc_labels[(arr_offset[:,0] == 0) & (arr_offset[:,1] != 0)] = doc_labels
        encoded_labels.append(doc_enc_labels.tolist())


    return encoded_labels

In [ ]:
# encoded X
X_train = tokenize(X_train_raw)
X_dev = tokenize(X_dev_raw)
X_test= tokenize(X_raw_test)

y_train = encode_tags(tags=y_train_raw, encodings=X_train, tag2id=tag2id)
y_dev = encode_tags(tags=y_dev_raw, encodings=X_dev, tag2id=tag2id)
y_test = encode_tags(tags=y_raw_test, encodings=X_test, tag2id=tag2id)


for X in [X_train, X_dev, X_test]:
  X.pop("offset_mapping")

In [ ]:
class CustomDataset(torch.utils.data.Dataset):
    """
    Pytorch dataset for easy batching
    """
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = CustomDataset(X_train, y_train)
val_dataset = CustomDataset(X_dev, y_dev)
test_dataset = CustomDataset(X_test,y_test)

In [ ]:
training_args = TrainingArguments(
    output_dir='/content/results/',   # output directory
    num_train_epochs=3,                   # total number of training epochs
    per_device_train_batch_size=16,       # batch size per device during training
    per_device_eval_batch_size=64,        # batch size for evaluation
    warmup_steps=500,                     # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                    # strength of weight decay
    logging_dir='/content/results',     # directory for storing logs
    logging_steps=10,
    evaluation_strategy = "epoch",
    #report_to = "wandb"
)

model = DistilBertForTokenClassification.from_pretrained(model_name, num_labels=len(unique_tags))

trainer = Trainer(
    model=model,                          # the instantiated 🤗 Transformers model to be trained
    args=training_args,                   # training arguments, defined above
    train_dataset=train_dataset,          # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    #compute_metrics = compute_metrics
)

trainer.train()
pred = trainer.predict(test_dataset)
preds = pred.predictions.argmax(-1)
#indices = val_index
#final_pred = pd.Series(preds).tolist()

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForTokenClassification: ['vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this 

Epoch,Training Loss,Validation Loss
1,0.507600,0.460651
2,0.214100,0.212497
3,0.088400,0.141092


In [ ]:
flat_list = [item for sublist in preds for item in sublist]

In [ ]:
np.shape(flat_list)

(55650,)

In [ ]:
flat_list1 = [item1 for sublist1 in y_test for item1 in sublist1]

In [ ]:
np.shape(flat_list1)

(55650,)

In [ ]:
lis_preds_frame = pd.DataFrame(flat_list)
lis_test_labels_frame = pd.DataFrame(flat_list1)
frame_combine_test = pd.concat([lis_preds_frame,lis_test_labels_frame],axis=1)
frame_combine_test.columns = ['preds','gold_data']

In [ ]:
frame_combine_test

,preds,gold_data
0,1,-100
1,1,1
2,1,1
3,1,1
4,1,1
...,...,...
55645,2,-100
55646,2,-100
55647,2,-100
55648,2,-100


In [ ]:
frame_combine_test_new = frame_combine_test[frame_combine_test.gold_data != -100]
frame_combine_test_new

,preds,gold_data
1,1,1
2,1,1
3,1,1
4,1,1
5,1,1
...,...,...
55505,2,2
55506,2,2
55507,2,2
55508,2,2


In [ ]:
print(id2tag)

{0: 'O', 1: 'E', 2: 'C'}


In [ ]:
targetname = ['O','E','C']

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()

              precision    recall  f1-score   support

           O       0.82      0.86      0.84       640
           E       0.93      0.94      0.93      5079
           C       0.95      0.94      0.95      6590

    accuracy                           0.93     12309
   macro avg       0.90      0.91      0.91     12309
weighted avg       0.93      0.93      0.93     12309



In [ ]:
#########10 runs #######################

In [ ]:

for i in range(10):
  trainer = Trainer(
    model=model,                          # the instantiated 🤗 Transformers model to be trained
    args=training_args,                   # training arguments, defined above
    train_dataset=train_dataset,          # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    #compute_metrics = compute_metrics
  )

  trainer.train()
  pred = trainer.predict(test_dataset)
  preds = pred.predictions.argmax(-1)
  flat_list = [item for sublist in preds for item in sublist]
  flat_list1 = [item1 for sublist1 in y_test for item1 in sublist1]

  lis_preds_frame = pd.DataFrame(flat_list)
  lis_test_labels_frame = pd.DataFrame(flat_list1)
  frame_combine_test = pd.concat([lis_preds_frame,lis_test_labels_frame],axis=1)
  frame_combine_test.columns = ['preds','gold_data']

  frame_combine_test_new = frame_combine_test[frame_combine_test.gold_data != -100]
  print(frame_combine_test_new)

  print(id2tag)
  targetname = ['E','C', 'O']


  print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()

Epoch,Training Loss,Validation Loss
1,0.448700,0.457309
2,0.186100,0.245943
3,0.079700,0.156811


       preds  gold_data
1          0          0
2          0          0
3          0          0
4          0          0
5          0          0
...      ...        ...
55505      1          1
55506      1          1
55507      1          1
55508      1          1
55509      1          1

[12309 rows x 2 columns]
{0: 'E', 1: 'C', 2: 'O'}
              precision    recall  f1-score   support

           E       0.93      0.93      0.93      5079
           C       0.94      0.95      0.95      6590
           O       0.86      0.88      0.87       640

    accuracy                           0.93     12309
   macro avg       0.91      0.92      0.91     12309
weighted avg       0.93      0.93      0.93     12309



Epoch,Training Loss,Validation Loss
1,0.044300,0.170389
2,0.042600,0.188150
3,0.062000,0.180282


       preds  gold_data
1          0          0
2          0          0
3          0          0
4          0          0
5          0          0
...      ...        ...
55505      1          1
55506      1          1
55507      1          1
55508      1          1
55509      1          1

[12309 rows x 2 columns]
{0: 'E', 1: 'C', 2: 'O'}
              precision    recall  f1-score   support

           E       0.94      0.94      0.94      5079
           C       0.95      0.96      0.95      6590
           O       0.89      0.86      0.88       640

    accuracy                           0.94     12309
   macro avg       0.93      0.92      0.92     12309
weighted avg       0.94      0.94      0.94     12309



Epoch,Training Loss,Validation Loss
1,0.011500,0.162106
2,0.011600,0.167159
3,0.005200,0.145856


...

       preds  gold_data
1          0          0
2          0          0
3          0          0
4          0          0
5          0          0
...      ...        ...
55505      1          1
55506      1          1
55507      1          1
55508      1          1
55509      1          1

[12309 rows x 2 columns]
{0: 'E', 1: 'C', 2: 'O'}
              precision    recall  f1-score   support

           E       0.92      0.94      0.93      5079
           C       0.96      0.94      0.95      6590
           O       0.81      0.88      0.84       640

    accuracy                           0.93     12309
   macro avg       0.90      0.92      0.91     12309
weighted avg       0.94      0.93      0.93     12309



Epoch,Training Loss,Validation Loss
1,0.000500,0.220552
2,0.012700,0.180328
3,0.003200,0.187042


       preds  gold_data
1          0          0
2          0          0
3          0          0
4          0          0
5          0          0
...      ...        ...
55505      1          1
55506      1          1
55507      1          1
55508      1          1
55509      1          1

[12309 rows x 2 columns]
{0: 'E', 1: 'C', 2: 'O'}
              precision    recall  f1-score   support

           E       0.94      0.94      0.94      5079
           C       0.95      0.95      0.95      6590
           O       0.83      0.90      0.86       640

    accuracy                           0.94     12309
   macro avg       0.91      0.93      0.92     12309
weighted avg       0.94      0.94      0.94     12309



Epoch,Training Loss,Validation Loss
1,0.001800,0.244628
2,0.002900,0.212958
3,0.005600,0.195478


       preds  gold_data
1          0          0
2          0          0
3          0          0
4          0          0
5          0          0
...      ...        ...
55505      1          1
55506      1          1
55507      1          1
55508      1          1
55509      1          1

[12309 rows x 2 columns]
{0: 'E', 1: 'C', 2: 'O'}
              precision    recall  f1-score   support

           E       0.93      0.93      0.93      5079
           C       0.96      0.94      0.95      6590
           O       0.77      0.91      0.83       640

    accuracy                           0.94     12309
   macro avg       0.89      0.93      0.91     12309
weighted avg       0.94      0.94      0.94     12309



In [ ]:
########Fincausal as train and SCITE as test#################################

In [ ]:
data_test = pd.read_excel('/content/test_data.xlsx',sheet_name = "Sheet3")
data_test.head(4)

,sentences,Bio_label
0,"[""Various"",""hormonal"","","",""bacterial"",""and"",""i...","[""C"",""C"",""C"",""C"",""C"",""C"",""C"",""O"",""E""]"
1,"[""A"",""stereo"",""buss"",""outputs"",""the"",""stereo"",...","[""C"",""C"",""C"",""O"",""E"",""E"",""E"",""E"",""O"",""C"",""C"",""..."
2,"[""The"",""incoming"",""water"",""caused"",""a"",""stain""...","[""C"",""C"",""C"",""O"",""E"",""E"",""O"",""O"",""O"",""O"",""O"",""..."
3,"[""The"",""genreal"",""anesthetic"",""cause"",""unconsc...","[""C"",""C"",""C"",""O"",""E"",""O"",""E"",""O"",""O"",""O"",""O"",""..."


In [ ]:
# load data and rename relevant columns to X (input) and y (output)
dataset_test = data_test[["sentences", "Bio_label"]].rename(columns={"sentences": "A", "Bio_label": "B"})
# load each row's data as a list of strings
X_raw_test, y_raw_test = [json.loads(tokens_test) for tokens_test in dataset_test.A.values], [json.loads(labels_test) for labels_test in dataset_test.B.values]

In [ ]:
for i, x in enumerate(y_raw_test):
    for j, a in enumerate(x):
        if 'CE' in a:
            y_raw_test[i][j] = a.replace('CE', 'O')

In [ ]:
y_raw_test

[['C', 'C', 'C', 'C', 'C', 'C', 'C', 'O', 'E'],
 ['C',
  'C',
  'C',
  'O',
  'E',
  'E',
  'E',
  'E',
  'O',
  'C',
  'C',
  'C',
  'O',
  'E',
  'E',
  'E',
  'E'],
 ['C',
  'C',
  'C',
  'O',
  'E',
  'E',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['C',
  'C',
  'C',
  'O',
  'E',
  'O',
  'E',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['C', 'C', 'C', 'O', 'O', 'E', 'O', 'E', 'O', 'O', 'O'],
 ['C', 'C', 'C', 'O', 'O', 'O', 'E', 'E', 'E'],
 ['C',
  'C',
  'C',
  'O',
  'O',
  'O',
  'E',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['C', 'C', 'C', 'O', 'O', 'O', 'O', 'O', 'E', 'E'],
 ['C',
  'C',
  'C',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'E',
  'E',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['C',
  'C',
  'C',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O

In [ ]:
X_test= tokenize(X_raw_test)

In [ ]:
y_test = encode_tags(tags=y_raw_test, encodings=X_test, tag2id=tag2id)

In [ ]:
test_dataset = CustomDataset(X_test,y_test)

In [ ]:
training_args = TrainingArguments(
    output_dir='/content/results/',   # output directory
    num_train_epochs=3,                   # total number of training epochs
    per_device_train_batch_size=16,       # batch size per device during training
    per_device_eval_batch_size=64,        # batch size for evaluation
    warmup_steps=500,                     # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                    # strength of weight decay
    logging_dir='/content/results',     # directory for storing logs
    logging_steps=10,
    evaluation_strategy = "epoch",
    #report_to = "wandb"
)

model = DistilBertForTokenClassification.from_pretrained(model_name, num_labels=len(unique_tags))

trainer = Trainer(
    model=model,                          # the instantiated 🤗 Transformers model to be trained
    args=training_args,                   # training arguments, defined above
    train_dataset=train_dataset,          # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    #compute_metrics = compute_metrics
)

trainer.train()
pred = trainer.predict(test_dataset)
preds = pred.predictions.argmax(-1)
#indices = val_index
#final_pred = pd.Series(preds).tolist()

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForTokenClassification: ['vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this 

Epoch,Training Loss,Validation Loss
1,0.521200,0.451448
2,0.228600,0.214891
3,0.091000,0.142699


In [ ]:
flat_list = [item for sublist in preds for item in sublist]

In [ ]:
np.shape(flat_list)

(13561,)

In [ ]:
flat_list1 = [item1 for sublist1 in y_test for item1 in sublist1]

In [ ]:
np.shape(flat_list1)

(13561,)

In [ ]:
lis_preds_frame = pd.DataFrame(flat_list)
lis_test_labels_frame = pd.DataFrame(flat_list1)
frame_combine_test = pd.concat([lis_preds_frame,lis_test_labels_frame],axis=1)
frame_combine_test.columns = ['preds','gold_data']

In [ ]:
frame_combine_test

,preds,gold_data
0,1,-100
1,2,2
2,2,2
3,2,-100
4,2,-100
...,...,...
13556,1,-100
13557,1,-100
13558,1,-100
13559,1,-100


In [ ]:
frame_combine_test_new = frame_combine_test[frame_combine_test.gold_data != -100]
frame_combine_test_new

,preds,gold_data
1,2,2
2,2,2
5,2,2
6,2,2
7,2,2
...,...,...
13519,2,2
13520,2,2
13521,2,0
13522,2,0


In [ ]:
print(id2tag)

{0: 'O', 1: 'E', 2: 'C'}


In [ ]:
targetname = ['O','E','C']

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()

              precision    recall  f1-score   support

           O       0.98      0.12      0.22      2688
           E       0.17      0.56      0.27       468
           C       0.24      0.92      0.38       465

    accuracy                           0.28      3621
   macro avg       0.47      0.53      0.29      3621
weighted avg       0.78      0.28      0.24      3621



In [ ]:
##########10 runs ######################################################

In [ ]:
for i in range(10):
  trainer = Trainer(
    model=model,                          # the instantiated 🤗 Transformers model to be trained
    args=training_args,                   # training arguments, defined above
    train_dataset=train_dataset,          # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    #compute_metrics = compute_metrics
  )

  trainer.train()
  pred = trainer.predict(test_dataset)
  preds = pred.predictions.argmax(-1)
  flat_list = [item for sublist in preds for item in sublist]
  flat_list1 = [item1 for sublist1 in y_test for item1 in sublist1]

  lis_preds_frame = pd.DataFrame(flat_list)
  lis_test_labels_frame = pd.DataFrame(flat_list1)
  frame_combine_test = pd.concat([lis_preds_frame,lis_test_labels_frame],axis=1)
  frame_combine_test.columns = ['preds','gold_data']

  frame_combine_test_new = frame_combine_test[frame_combine_test.gold_data != -100]
  print(frame_combine_test_new)

  print(id2tag)
  targetname = ['E','C', 'O']


  print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()

Epoch,Training Loss,Validation Loss
1,0.001000,0.182497
2,0.007600,0.261587
3,0.002300,0.158455


       preds  gold_data
1          1          1
2          1          1
5          1          1
6          1          1
7          1          1
...      ...        ...
13519      1          1
13520      1          1
13521      1          2
13522      1          2
13523      1          2

[3621 rows x 2 columns]
{0: 'E', 1: 'C', 2: 'O'}
              precision    recall  f1-score   support

           E       0.20      0.74      0.32       468
           C       0.28      0.91      0.42       465
           O       1.00      0.14      0.25      2688

    accuracy                           0.32      3621
   macro avg       0.49      0.60      0.33      3621
weighted avg       0.80      0.32      0.28      3621



Epoch,Training Loss,Validation Loss
1,0.002900,0.201465
2,0.091100,0.183356
3,0.001400,0.198767


       preds  gold_data
1          1          1
2          1          1
5          1          1
6          1          1
7          1          1
...      ...        ...
13519      1          1
13520      1          1
13521      1          2
13522      1          2
13523      1          2

[3621 rows x 2 columns]
{0: 'E', 1: 'C', 2: 'O'}
              precision    recall  f1-score   support

           E       0.19      0.61      0.29       468
           C       0.24      0.91      0.38       465
           O       1.00      0.13      0.24      2688

    accuracy                           0.29      3621
   macro avg       0.48      0.55      0.30      3621
weighted avg       0.80      0.29      0.26      3621



Epoch,Training Loss,Validation Loss
1,0.000100,0.228348
2,0.003600,0.141874
3,0.004200,0.175797


...

       preds  gold_data
1          1          1
2          1          1
5          1          1
6          1          1
7          1          1
...      ...        ...
13519      1          1
13520      1          1
13521      1          2
13522      1          2
13523      1          2

[3621 rows x 2 columns]
{0: 'E', 1: 'C', 2: 'O'}
              precision    recall  f1-score   support

           E       0.19      0.67      0.30       468
           C       0.26      0.89      0.40       465
           O       0.99      0.14      0.25      2688

    accuracy                           0.31      3621
   macro avg       0.48      0.57      0.31      3621
weighted avg       0.80      0.31      0.27      3621



Epoch,Training Loss,Validation Loss
1,0.003300,0.244719
2,0.013200,0.242150
3,0.001100,0.187149


       preds  gold_data
1          1          1
2          1          1
5          1          1
6          1          1
7          1          1
...      ...        ...
13519      1          1
13520      1          1
13521      1          2
13522      1          2
13523      1          2

[3621 rows x 2 columns]
{0: 'E', 1: 'C', 2: 'O'}
              precision    recall  f1-score   support

           E       0.19      0.78      0.31       468
           C       0.29      0.82      0.43       465
           O       0.99      0.16      0.28      2688

    accuracy                           0.32      3621
   macro avg       0.49      0.58      0.34      3621
weighted avg       0.80      0.32      0.30      3621



Epoch,Training Loss,Validation Loss
1,0.000100,0.194846
2,0.000400,0.219317
3,0.000700,0.194883


       preds  gold_data
1          1          1
2          1          1
5          1          1
6          1          1
7          1          1
...      ...        ...
13519      1          1
13520      1          1
13521      1          2
13522      1          2
13523      1          2

[3621 rows x 2 columns]
{0: 'E', 1: 'C', 2: 'O'}
              precision    recall  f1-score   support

           E       0.19      0.74      0.30       468
           C       0.27      0.77      0.40       465
           O       0.99      0.16      0.28      2688

    accuracy                           0.32      3621
   macro avg       0.48      0.56      0.33      3621
weighted avg       0.79      0.32      0.30      3621



In [ ]:
###################Fincausal as train and Organizational as test##############################

In [ ]:
# mount Google Drive
from google.colab import drive
import os

drive.mount("/content/drive", force_remount=True)

# data location
#parent_dir = "/content/drive/MyDrive"
parent_dir = "/content/drive/MyDrive/"
mk_data_path = lambda filename: os.path.join(parent_dir, filename)
#data_path = f"{parent_dir}/BERT_data_final.xlsx"
data_path = mk_data_path("BERT_data_final1.xlsx")

# once finished, flush changes to files to make them visible (assuming we write to Google Drive)
# drive.flush_and_unmount()

Mounted at /content/drive


In [ ]:
data = pd.read_excel(data_path)
data.head(4)

,Causal-relation text,BIO Code,"{""words"":Causal-relation text,""ner"":BIO Code}",BIO_Code_changed,BIO Code-changed-NER
0,"[""When "",""a "",""policyholder "",""or "",""insured ""...","[""O"",""O"",""B-C"",""I-C"",""I-C"",""I-C"",""I-C"",""I-C"",""...","{""words"":[""When "",""a "",""policyholder "",""or "",""...","[""O"",""O"",""C"",""C"",""C"",""C"",""C"",""C"",""C"",""O"",""O"",""...","[""O"",""O"",""B-ENTITY"",""B-ENTITY"",""B-ENTITY"",""B-E..."
1,"[""During "",""2020 "",""in "",""response "",""to "",""th...","[""O"",""O"",""O"",""O"",""O"",""B-C"",""I-C"",""I-C"",""I-C"",""...","{""words"":[""During "",""2020 "",""in "",""response "",...","[""O"",""O"",""O"",""O"",""O"",""C"",""C"",""C"",""C"",""C"",""C"",""...","[""O"",""O"",""O"",""O"",""O"",""B-ENTITY"",""B-ENTITY"",""B-..."
2,"[""Prolonged "",""periods "",""of "",""low "",""interes...","[""B-C"",""I-C"",""I-C"",""I-C"",""I-C"",""I-C"",""O"",""B-CT...","{""words"":[""Prolonged "",""periods "",""of "",""low ""...","[""C"",""C"",""C"",""C"",""C"",""C"",""O"",""CT"",""E"",""E"",""O"",...","[""B-ENTITY"",""B-ENTITY"",""B-ENTITY"",""B-ENTITY"",""..."
3,"[""Conversely "",""a "",""rise "",""in "",""interest "",...","[""O"",""O"",""B-C"",""I-C"",""I-C"",""I-C"",""O"",""B-CT"",""O...","{""words"":[""Conversely "",""a "",""rise "",""in "",""in...","[""O"",""O"",""C"",""C"",""C"",""C"",""O"",""CT"",""O"",""O"",""O"",...","[""O"",""O"",""B-ENTITY"",""B-ENTITY"",""B-ENTITY"",""B-E..."


In [ ]:
# load data and rename relevant columns to X (input) and y (output)
dataset = data[["Causal-relation text", "BIO_Code_changed"]].rename(columns={"Causal-relation text": "X", "BIO_Code_changed": "y"})
# load each row's data as a list of strings
X_raw, y_raw = [json.loads(tokens) for tokens in dataset.X.values], [json.loads(labels) for labels in dataset.y.values]

In [ ]:
# # Our data is split into sentences


# # FIXME: we also need test data!
# X_train_raw, X_dev_raw, y_train_raw, y_dev_raw = train_test_split(X_raw, y_raw, test_size=.2, random_state=RANDOM_SEED)

In [ ]:
X_test= tokenize(X_raw)

In [ ]:
for i, x in enumerate(y_raw):
    for j, a in enumerate(x):
        if 'CT' in a:
            y_raw[i][j] = a.replace('CT', 'O')

In [ ]:
y_test = encode_tags(tags=y_raw, encodings=X_test, tag2id=tag2id)

In [ ]:
test_dataset = CustomDataset(X_test,y_test)

In [ ]:
training_args = TrainingArguments(
    output_dir='/content/results/',   # output directory
    num_train_epochs=3,                   # total number of training epochs
    per_device_train_batch_size=16,       # batch size per device during training
    per_device_eval_batch_size=64,        # batch size for evaluation
    warmup_steps=500,                     # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                    # strength of weight decay
    logging_dir='/content/results',     # directory for storing logs
    logging_steps=10,
    evaluation_strategy = "epoch",
    #report_to = "wandb"
)

model = DistilBertForTokenClassification.from_pretrained(model_name, num_labels=len(unique_tags))

trainer = Trainer(
    model=model,                          # the instantiated 🤗 Transformers model to be trained
    args=training_args,                   # training arguments, defined above
    train_dataset=train_dataset,          # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    #compute_metrics = compute_metrics
)

trainer.train()
pred = trainer.predict(test_dataset)
preds = pred.predictions.argmax(-1)
#indices = val_index
#final_pred = pd.Series(preds).tolist()

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForTokenClassification: ['vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this 

Epoch,Training Loss,Validation Loss
1,0.521200,0.451448
2,0.228600,0.214891
3,0.091000,0.142699


In [ ]:
flat_list = [item for sublist in preds for item in sublist]

In [ ]:
np.shape(flat_list)

(52746,)

In [ ]:
flat_list1 = [item1 for sublist1 in y_test for item1 in sublist1]

In [ ]:
np.shape(flat_list1)

(52746,)

In [ ]:
lis_preds_frame = pd.DataFrame(flat_list)
lis_test_labels_frame = pd.DataFrame(flat_list1)
frame_combine_test = pd.concat([lis_preds_frame,lis_test_labels_frame],axis=1)
frame_combine_test.columns = ['preds','gold_data']

In [ ]:
frame_combine_test

,preds,gold_data
0,1,-100
1,1,2
2,1,2
3,1,2
4,1,2
...,...,...
52741,1,-100
52742,1,-100
52743,1,-100
52744,1,-100


In [ ]:
frame_combine_test_new = frame_combine_test[frame_combine_test.gold_data != -100]
frame_combine_test_new

,preds,gold_data
1,1,2
2,1,2
3,1,2
4,1,2
5,1,2
...,...,...
52655,2,1
52656,2,1
52657,2,1
52658,2,1


In [ ]:
print(id2tag)

{0: 'O', 1: 'E', 2: 'C'}


In [ ]:
targetname = ['O','E','C']

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()

              precision    recall  f1-score   support

           O       0.82      0.26      0.40      2978
           E       0.64      0.74      0.68      5662
           C       0.63      0.76      0.69      5694

    accuracy                           0.65     14334
   macro avg       0.70      0.59      0.59     14334
weighted avg       0.67      0.65      0.63     14334



In [ ]:
##############10 runs###################################################

In [ ]:
training_args = TrainingArguments(
    output_dir='/content/results/',   # output directory
    num_train_epochs=3,                   # total number of training epochs
    per_device_train_batch_size=16,       # batch size per device during training
    per_device_eval_batch_size=64,        # batch size for evaluation
    warmup_steps=500,                     # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                    # strength of weight decay
    logging_dir='/content/results',     # directory for storing logs
    logging_steps=10,
    evaluation_strategy = "epoch",
    #report_to = "wandb"
)

model = DistilBertForTokenClassification.from_pretrained(model_name, num_labels=len(unique_tags))


model.safetensors:   0%|          | 0.00/263M [00:00<?, ?B/s]

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from sklearn.metrics import classification_report

In [ ]:
for i in range(10):
  trainer = Trainer(
    model=model,                          # the instantiated 🤗 Transformers model to be trained
    args=training_args,                   # training arguments, defined above
    train_dataset=train_dataset,          # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    #compute_metrics = compute_metrics
  )

  trainer.train()
  pred = trainer.predict(test_dataset)
  preds = pred.predictions.argmax(-1)
  flat_list = [item for sublist in preds for item in sublist]
  flat_list1 = [item1 for sublist1 in y_test for item1 in sublist1]

  lis_preds_frame = pd.DataFrame(flat_list)
  lis_test_labels_frame = pd.DataFrame(flat_list1)
  frame_combine_test = pd.concat([lis_preds_frame,lis_test_labels_frame],axis=1)
  frame_combine_test.columns = ['preds','gold_data']

  frame_combine_test_new = frame_combine_test[frame_combine_test.gold_data != -100]
  print(frame_combine_test_new)

  print(id2tag)
  targetname = ['E','C', 'O']


  print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()

Epoch,Training Loss,Validation Loss
1,0.479200,0.456639
2,0.201200,0.228352
3,0.082500,0.160798


        preds  gold_data
1           1          2
2           1          2
3           1          1
5           1          1
6           1          1
...       ...        ...
491283      1          1
491284      1          1
491285      1          1
491286      1          1
491287      1          1

[73283 rows x 2 columns]
{0: 'E', 1: 'C', 2: 'O'}
              precision    recall  f1-score   support

           E       0.63      0.78      0.70     28547
           C       0.63      0.73      0.68     28260
           O       0.79      0.25      0.38     16476

    accuracy                           0.64     73283
   macro avg       0.68      0.59      0.59     73283
weighted avg       0.67      0.64      0.62     73283



Epoch,Training Loss,Validation Loss
1,0.083700,0.176610
2,0.013600,0.193250
3,0.021200,0.126128


        preds  gold_data
1           1          2
2           1          2
3           1          1
5           1          1
6           1          1
...       ...        ...
491283      1          1
491284      1          1
491285      1          1
491286      1          1
491287      1          1

[73283 rows x 2 columns]
{0: 'E', 1: 'C', 2: 'O'}
              precision    recall  f1-score   support

           E       0.66      0.70      0.68     28547
           C       0.60      0.81      0.69     28260
           O       0.83      0.24      0.38     16476

    accuracy                           0.64     73283
   macro avg       0.70      0.58      0.58     73283
weighted avg       0.68      0.64      0.61     73283



Epoch,Training Loss,Validation Loss
1,0.017500,0.151987
2,0.076100,0.185538
3,0.008600,0.161925


...

        preds  gold_data
1           1          2
2           1          2
3           1          1
5           1          1
6           1          1
...       ...        ...
491283      1          1
491284      1          1
491285      1          1
491286      1          1
491287      1          1

[73283 rows x 2 columns]
{0: 'E', 1: 'C', 2: 'O'}
              precision    recall  f1-score   support

           E       0.65      0.76      0.70     28547
           C       0.63      0.78      0.70     28260
           O       0.84      0.24      0.38     16476

    accuracy                           0.65     73283
   macro avg       0.71      0.60      0.59     73283
weighted avg       0.69      0.65      0.63     73283



Epoch,Training Loss,Validation Loss
1,0.000300,0.203886
2,0.005800,0.249336
3,0.004100,0.201099


        preds  gold_data
1           1          2
2           1          2
3           1          1
5           1          1
6           1          1
...       ...        ...
491283      1          1
491284      1          1
491285      1          1
491286      1          1
491287      1          1

[73283 rows x 2 columns]
{0: 'E', 1: 'C', 2: 'O'}
              precision    recall  f1-score   support

           E       0.63      0.82      0.71     28547
           C       0.66      0.74      0.70     28260
           O       0.87      0.24      0.38     16476

    accuracy                           0.66     73283
   macro avg       0.72      0.60      0.60     73283
weighted avg       0.70      0.66      0.63     73283



Epoch,Training Loss,Validation Loss
1,0.032100,0.188584
2,0.006600,0.219089
3,0.010400,0.180988


        preds  gold_data
1           1          2
2           1          2
3           1          1
5           1          1
6           1          1
...       ...        ...
491283      1          1
491284      1          1
491285      1          1
491286      1          1
491287      1          1

[73283 rows x 2 columns]
{0: 'E', 1: 'C', 2: 'O'}
              precision    recall  f1-score   support

           E       0.64      0.80      0.71     28547
           C       0.64      0.75      0.69     28260
           O       0.87      0.22      0.35     16476

    accuracy                           0.65     73283
   macro avg       0.72      0.59      0.58     73283
weighted avg       0.69      0.65      0.62     73283



In [ ]:
#################### Organizational data as train and organizational data as test######################

In [ ]:
# mount Google Drive
from google.colab import drive
import os

drive.mount("/content/drive", force_remount=True)

# data location
#parent_dir = "/content/drive/MyDrive"
parent_dir = "/content/drive/MyDrive/"
mk_data_path = lambda filename: os.path.join(parent_dir, filename)
#data_path = f"{parent_dir}/BERT_data_final.xlsx"
data_path = mk_data_path("BERT_data_final1.xlsx")

# once finished, flush changes to files to make them visible (assuming we write to Google Drive)
# drive.flush_and_unmount()

Mounted at /content/drive


In [ ]:
data = pd.read_excel(data_path)
data.head(4)

,Causal-relation text,BIO Code,"{""words"":Causal-relation text,""ner"":BIO Code}",BIO_Code_changed,BIO Code-changed-NER
0,"[""When "",""a "",""policyholder "",""or "",""insured ""...","[""O"",""O"",""B-C"",""I-C"",""I-C"",""I-C"",""I-C"",""I-C"",""...","{""words"":[""When "",""a "",""policyholder "",""or "",""...","[""O"",""O"",""C"",""C"",""C"",""C"",""C"",""C"",""C"",""O"",""O"",""...","[""O"",""O"",""B-ENTITY"",""B-ENTITY"",""B-ENTITY"",""B-E..."
1,"[""During "",""2020 "",""in "",""response "",""to "",""th...","[""O"",""O"",""O"",""O"",""O"",""B-C"",""I-C"",""I-C"",""I-C"",""...","{""words"":[""During "",""2020 "",""in "",""response "",...","[""O"",""O"",""O"",""O"",""O"",""C"",""C"",""C"",""C"",""C"",""C"",""...","[""O"",""O"",""O"",""O"",""O"",""B-ENTITY"",""B-ENTITY"",""B-..."
2,"[""Prolonged "",""periods "",""of "",""low "",""interes...","[""B-C"",""I-C"",""I-C"",""I-C"",""I-C"",""I-C"",""O"",""B-CT...","{""words"":[""Prolonged "",""periods "",""of "",""low ""...","[""C"",""C"",""C"",""C"",""C"",""C"",""O"",""CT"",""E"",""E"",""O"",...","[""B-ENTITY"",""B-ENTITY"",""B-ENTITY"",""B-ENTITY"",""..."
3,"[""Conversely "",""a "",""rise "",""in "",""interest "",...","[""O"",""O"",""B-C"",""I-C"",""I-C"",""I-C"",""O"",""B-CT"",""O...","{""words"":[""Conversely "",""a "",""rise "",""in "",""in...","[""O"",""O"",""C"",""C"",""C"",""C"",""O"",""CT"",""O"",""O"",""O"",...","[""O"",""O"",""B-ENTITY"",""B-ENTITY"",""B-ENTITY"",""B-E..."


In [ ]:
# load data and rename relevant columns to X (input) and y (output)
dataset = data[["Causal-relation text", "BIO_Code_changed"]].rename(columns={"Causal-relation text": "X", "BIO_Code_changed": "y"})
# load each row's data as a list of strings
X_raw, y_raw = [json.loads(tokens) for tokens in dataset.X.values], [json.loads(labels) for labels in dataset.y.values]

In [ ]:
unique_tags = set(label for labels in y_raw for label in labels)
tag2id = {tag: id for id, tag in enumerate(unique_tags)}
id2tag = {id: tag for tag, id in tag2id.items()}

In [ ]:
print(id2tag)

{0: 'C', 1: 'CT', 2: 'O', 3: 'E'}


In [ ]:
# Our data is split into sentences


# FIXME: we also need test data!
X_train_raw, X_dev_raw, y_train_raw, y_dev_raw = train_test_split(X_raw, y_raw, test_size=.2, random_state=RANDOM_SEED)

In [ ]:
X_train_final = np.array(X_train_raw)
y_train_final = np.array(y_train_raw)

<ipython-input-11-100920d9c2b5>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_train_final = np.array(X_train_raw)
<ipython-input-11-100920d9c2b5>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  y_train_final = np.array(y_train_raw)


In [ ]:
print(np.shape(X_dev_raw))
print(np.shape(y_dev_raw))
print(np.shape(X_train_raw))

(447,)
(447,)
(1787,)


/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:2009: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = asarray(a).shape


In [ ]:
import os
k=2
seed = 1
kf = KFold(n_splits=k,random_state=seed, shuffle=True)
metrics=[]
i=0
for train_index, val_index in kf.split(X_train_final,y_train_final):
  train_texts, test_texts =X_train_final[train_index].tolist(),X_train_final[val_index].tolist()
  train_labels, test_labels = y_train_final[train_index].tolist(),y_train_final[val_index].tolist()

In [ ]:
# ensure our data is well-formed:
for ds in [X_train_raw, X_train_raw]:
  for row in ds:
    #print(row)
    assert all(len(tok) > 0 for tok in row)

In [ ]:
# ensure our data is well-formed:
for ds in [test_texts, test_texts]:
  for row in ds:
    assert all(len(tok) > 0 for tok in row)

In [ ]:
# ensure our data is well-formed:
for ds in [train_texts, train_texts]:
  for row in ds:
    assert all(len(tok) > 0 for tok in row)

In [ ]:
def encode_tags(tags, encodings, tag2id):
    """
    See https://huggingface.co/transformers/custom_datasets.html#tok-ner

    >If the tokenizer splits a token into multiple sub-tokens, then we will end up with a mismatch between our tokens and our labels.
    >One way to handle this is to only train on the tag labels for the first subtoken of a split token.
    >We can do this in 🤗 Transformers by setting the labels we wish to ignore to -100.
    >In the example above, if the label for @HuggingFace is 3 (indexing `B-corporation`),
    >we would set the labels of ['@', 'hugging', '##face'] to [3, -100, -100].
    """
    labels = [[tag2id[tag] for tag in doc] for doc in tags]
    encoded_labels = []
    for doc_labels, doc_offset in zip(labels, encodings.offset_mapping):
        # create an empty array of -100
        doc_enc_labels = np.ones(len(doc_offset),dtype=int) * -100
        arr_offset = np.array(doc_offset)
        # print(arr_offset)
        # print(type(arr_offset))

        # set labels whose first offset position is 0 and the second is not 0
        doc_enc_labels[(arr_offset[:,0] == 0) & (arr_offset[:,1] != 0)] = doc_labels
        encoded_labels.append(doc_enc_labels.tolist())

    return encoded_labels

In [ ]:
# encoded X
X_train = tokenize(train_texts)
X_dev = tokenize(X_dev_raw)
X_test= tokenize(test_texts)

y_train = encode_tags(tags=train_labels, encodings=X_train, tag2id=tag2id)
y_dev = encode_tags(tags=y_dev_raw, encodings=X_dev, tag2id=tag2id)
y_test = encode_tags(tags=test_labels, encodings=X_test, tag2id=tag2id)


for X in [X_train, X_dev, X_test]:
  X.pop("offset_mapping")

In [ ]:
class CustomDataset(torch.utils.data.Dataset):
    """
    Pytorch dataset for easy batching
    """
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = CustomDataset(X_train, y_train)
val_dataset = CustomDataset(X_dev, y_dev)
test_dataset = CustomDataset(X_test,y_test)

In [ ]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=y_train)

<ipython-input-20-2ab7c11a097d>:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


In [ ]:
training_args = TrainingArguments(
    output_dir='/content/results/',   # output directory
    num_train_epochs=3,                   # total number of training epochs
    per_device_train_batch_size=16,       # batch size per device during training
    per_device_eval_batch_size=64,        # batch size for evaluation
    warmup_steps=500,                     # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                    # strength of weight decay
    logging_dir='/content/results',     # directory for storing logs
    logging_steps=10,
    evaluation_strategy = "epoch",
    #report_to = "wandb"
)

model = DistilBertForTokenClassification.from_pretrained(model_name, num_labels=len(unique_tags))

trainer = Trainer(
    model=model,                          # the instantiated 🤗 Transformers model to be trained
    args=training_args,                   # training arguments, defined above
    train_dataset=train_dataset,          # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    #compute_metrics = compute_metrics
)

trainer.train()
pred = trainer.predict(test_dataset)
preds = pred.predictions.argmax(-1)
#indices = val_index
#final_pred = pd.Series(preds).tolist()

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForTokenClassification: ['vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this 

Epoch,Training Loss,Validation Loss
1,1.214300,1.153262
2,0.914300,0.878064
3,0.633900,0.537422


In [ ]:
flat_list = [item for sublist in preds for item in sublist]

In [ ]:
np.shape(flat_list)

(118769,)

In [ ]:
flat_list1 = [item1 for sublist1 in y_test for item1 in sublist1]

In [ ]:
np.shape(flat_list1)

(118769,)

In [ ]:
lis_preds_frame = pd.DataFrame(flat_list)
lis_test_labels_frame = pd.DataFrame(flat_list1)
frame_combine_test = pd.concat([lis_preds_frame,lis_test_labels_frame],axis=1)
frame_combine_test.columns = ['preds','gold_data']

In [ ]:
frame_combine_test

,preds,gold_data
0,3,-100
1,2,2
2,2,-100
3,2,-100
4,2,-100
...,...,...
118764,2,-100
118765,3,-100
118766,3,-100
118767,3,-100


In [ ]:
frame_combine_test_new = frame_combine_test[frame_combine_test.gold_data != -100]
frame_combine_test_new

,preds,gold_data
1,2,2
5,2,2
6,2,2
7,2,2
8,2,2
...,...,...
118667,0,0
118668,0,1
118669,3,3
118670,3,3


In [ ]:
print(id2tag)

{0: 'CT', 1: 'O', 2: 'E', 3: 'C'}


In [ ]:
targetname = ['CT', 'O', 'E', 'C']

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()

              precision    recall  f1-score   support

          CT       0.86      0.94      0.90      1780
           O       0.66      0.25      0.36      5216
           E       0.77      0.94      0.84     11316
           C       0.79      0.83      0.81     11362

    accuracy                           0.78     29674
   macro avg       0.77      0.74      0.73     29674
weighted avg       0.76      0.78      0.75     29674



In [ ]:
training_args = TrainingArguments(
    output_dir='/content/results/',   # output directory
    num_train_epochs=3,                   # total number of training epochs
    per_device_train_batch_size=16,       # batch size per device during training
    per_device_eval_batch_size=64,        # batch size for evaluation
    warmup_steps=500,                     # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                    # strength of weight decay
    logging_dir='/content/results',     # directory for storing logs
    logging_steps=10,
    evaluation_strategy = "epoch",
    #report_to = "wandb"
)

model = DistilBertForTokenClassification.from_pretrained(model_name, num_labels=len(unique_tags))

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
print(id2tag)

{0: 'C', 1: 'CT', 2: 'O', 3: 'E'}


In [ ]:
from sklearn.metrics import classification_report

In [ ]:
for i in range(10):
  trainer = Trainer(
    model=model,                          # the instantiated 🤗 Transformers model to be trained
    args=training_args,                   # training arguments, defined above
    train_dataset=train_dataset,          # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    #compute_metrics = compute_metrics
  )

  trainer.train()
  pred = trainer.predict(test_dataset)
  preds = pred.predictions.argmax(-1)
  flat_list = [item for sublist in preds for item in sublist]
  flat_list1 = [item1 for sublist1 in y_test for item1 in sublist1]

  lis_preds_frame = pd.DataFrame(flat_list)
  lis_test_labels_frame = pd.DataFrame(flat_list1)
  frame_combine_test = pd.concat([lis_preds_frame,lis_test_labels_frame],axis=1)
  frame_combine_test.columns = ['preds','gold_data']

  frame_combine_test_new = frame_combine_test[frame_combine_test.gold_data != -100]
  print(frame_combine_test_new)

  print(id2tag)
  targetname = ['C','CT', 'O','E']


  print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()

Epoch,Training Loss,Validation Loss
1,0.463500,0.484968
2,0.383400,0.479423
3,0.368600,0.459571


        preds  gold_data
1           3          3
5           3          3
6           3          3
7           3          3
8           3          3
...       ...        ...
118667      1          1
118668      2          2
118669      0          0
118670      0          0
118671      0          0

[29674 rows x 2 columns]
{0: 'C', 1: 'CT', 2: 'O', 3: 'E'}
              precision    recall  f1-score   support

           C       0.78      0.87      0.82     11362
          CT       0.93      0.94      0.93      1780
           O       0.57      0.40      0.47      5216
           E       0.85      0.87      0.86     11316

    accuracy                           0.79     29674
   macro avg       0.78      0.77      0.77     29674
weighted avg       0.78      0.79      0.78     29674



Epoch,Training Loss,Validation Loss
1,0.287200,0.461732
2,0.230800,0.508077
3,0.224300,0.510377


        preds  gold_data
1           3          3
5           3          3
6           3          3
7           3          3
8           3          3
...       ...        ...
118667      1          1
118668      2          2
118669      0          0
118670      0          0
118671      0          0

[29674 rows x 2 columns]
{0: 'C', 1: 'CT', 2: 'O', 3: 'E'}
              precision    recall  f1-score   support

           C       0.76      0.89      0.82     11362
          CT       0.94      0.95      0.94      1780
           O       0.55      0.42      0.48      5216
           E       0.88      0.83      0.85     11316

    accuracy                           0.79     29674
   macro avg       0.78      0.77      0.77     29674
weighted avg       0.78      0.79      0.78     29674



Epoch,Training Loss,Validation Loss
1,0.180000,0.516172
2,0.126500,0.577502
3,0.138900,0.615741


...

        preds  gold_data
1           3          3
5           3          3
6           3          3
7           3          3
8           3          3
...       ...        ...
118667      1          1
118668      2          2
118669      0          0
118670      0          0
118671      0          0

[29674 rows x 2 columns]
{0: 'C', 1: 'CT', 2: 'O', 3: 'E'}
              precision    recall  f1-score   support

           C       0.80      0.83      0.81     11362
          CT       0.94      0.96      0.95      1780
           O       0.56      0.45      0.50      5216
           E       0.84      0.89      0.86     11316

    accuracy                           0.79     29674
   macro avg       0.79      0.78      0.78     29674
weighted avg       0.78      0.79      0.79     29674



Epoch,Training Loss,Validation Loss
1,0.014300,0.954751
2,0.004200,0.999999
3,0.012400,1.046583


        preds  gold_data
1           3          3
5           3          3
6           3          3
7           3          3
8           3          3
...       ...        ...
118667      1          1
118668      2          2
118669      0          0
118670      0          0
118671      0          0

[29674 rows x 2 columns]
{0: 'C', 1: 'CT', 2: 'O', 3: 'E'}
              precision    recall  f1-score   support

           C       0.80      0.84      0.82     11362
          CT       0.93      0.96      0.94      1780
           O       0.55      0.45      0.50      5216
           E       0.86      0.88      0.87     11316

    accuracy                           0.79     29674
   macro avg       0.78      0.78      0.78     29674
weighted avg       0.79      0.79      0.79     29674



Epoch,Training Loss,Validation Loss
1,0.011700,1.059433
2,0.002500,1.071217
3,0.012800,1.092427


        preds  gold_data
1           3          3
5           3          3
6           3          3
7           3          3
8           3          3
...       ...        ...
118667      1          1
118668      2          2
118669      0          0
118670      0          0
118671      0          0

[29674 rows x 2 columns]
{0: 'C', 1: 'CT', 2: 'O', 3: 'E'}
              precision    recall  f1-score   support

           C       0.80      0.83      0.82     11362
          CT       0.94      0.96      0.95      1780
           O       0.59      0.44      0.50      5216
           E       0.84      0.90      0.87     11316

    accuracy                           0.80     29674
   macro avg       0.79      0.78      0.78     29674
weighted avg       0.79      0.80      0.79     29674



In [ ]:
######################Organizational data as train and SCITE as test##################

In [ ]:
data_test = pd.read_excel('/content/test_data.xlsx',sheet_name = "Sheet3")
data_test.head(4)

,sentences,Bio_label
0,"[""Various"",""hormonal"","","",""bacterial"",""and"",""i...","[""C"",""C"",""C"",""C"",""C"",""C"",""C"",""O"",""E""]"
1,"[""A"",""stereo"",""buss"",""outputs"",""the"",""stereo"",...","[""C"",""C"",""C"",""O"",""E"",""E"",""E"",""E"",""O"",""C"",""C"",""..."
2,"[""The"",""incoming"",""water"",""caused"",""a"",""stain""...","[""C"",""C"",""C"",""O"",""E"",""E"",""O"",""O"",""O"",""O"",""O"",""..."
3,"[""The"",""genreal"",""anesthetic"",""cause"",""unconsc...","[""C"",""C"",""C"",""O"",""E"",""O"",""E"",""O"",""O"",""O"",""O"",""..."


In [ ]:
# load data and rename relevant columns to X (input) and y (output)
dataset_test = data_test[["sentences", "Bio_label"]].rename(columns={"sentences": "A", "Bio_label": "B"})
# load each row's data as a list of strings
X_raw_test, y_raw_test = [json.loads(tokens_test) for tokens_test in dataset_test.A.values], [json.loads(labels_test) for labels_test in dataset_test.B.values]

In [ ]:
for i, x in enumerate(y_raw_test):
    for j, a in enumerate(x):
        if 'CE' in a:
            y_raw_test[i][j] = a.replace('CE', 'CT')

In [ ]:
X_test= tokenize(X_raw_test)

In [ ]:
y_test = encode_tags(tags=y_raw_test, encodings=X_test, tag2id=tag2id)

In [ ]:
test_dataset = CustomDataset(X_test,y_test)

In [ ]:
print(id2tag)

{0: 'C', 1: 'CT', 2: 'O', 3: 'E'}


In [ ]:
for i in range(10):
  trainer = Trainer(
    model=model,                          # the instantiated 🤗 Transformers model to be trained
    args=training_args,                   # training arguments, defined above
    train_dataset=train_dataset,          # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    #compute_metrics = compute_metrics
  )

  trainer.train()
  pred = trainer.predict(test_dataset)
  preds = pred.predictions.argmax(-1)
  flat_list = [item for sublist in preds for item in sublist]
  flat_list1 = [item1 for sublist1 in y_test for item1 in sublist1]

  lis_preds_frame = pd.DataFrame(flat_list)
  lis_test_labels_frame = pd.DataFrame(flat_list1)
  frame_combine_test = pd.concat([lis_preds_frame,lis_test_labels_frame],axis=1)
  frame_combine_test.columns = ['preds','gold_data']

  frame_combine_test_new = frame_combine_test[frame_combine_test.gold_data != -100]
  print(frame_combine_test_new)

  print(id2tag)
  targetname = ['C','CT', 'O', 'E']


  print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()

Epoch,Training Loss,Validation Loss
1,0.007300,1.087650
2,0.001100,1.139605
3,0.003500,1.208249


       preds  gold_data
1          2          0
2          0          0
5          0          0
6          0          0
7          0          0
...      ...        ...
13519      3          0
13520      0          0
13521      0          2
13522      0          2
13523      0          2

[3621 rows x 2 columns]
{0: 'C', 1: 'CT', 2: 'O', 3: 'E'}
              precision    recall  f1-score   support

           C       0.24      0.74      0.36       465
          CT       0.00      0.00      0.00        25
           O       0.81      0.26      0.39      2663
           E       0.21      0.53      0.30       468

    accuracy                           0.35      3621
   macro avg       0.31      0.38      0.26      3621
weighted avg       0.65      0.35      0.37      3621



Epoch,Training Loss,Validation Loss
1,0.005400,1.189015
2,0.002200,1.166711
3,0.002800,1.219783


       preds  gold_data
1          2          0
2          0          0
5          0          0
6          0          0
7          0          0
...      ...        ...
13519      2          0
13520      0          0
13521      0          2
13522      0          2
13523      0          2

[3621 rows x 2 columns]
{0: 'C', 1: 'CT', 2: 'O', 3: 'E'}
              precision    recall  f1-score   support

           C       0.24      0.78      0.36       465
          CT       0.00      0.00      0.00        25
           O       0.82      0.27      0.40      2663
           E       0.22      0.52      0.31       468

    accuracy                           0.36      3621
   macro avg       0.32      0.39      0.27      3621
weighted avg       0.66      0.36      0.38      3621



Epoch,Training Loss,Validation Loss
1,0.003100,1.179819
2,0.000500,1.199186
3,0.002100,1.242707


...

       preds  gold_data
1          2          0
2          0          0
5          0          0
6          0          0
7          0          0
...      ...        ...
13519      3          0
13520      0          0
13521      0          2
13522      0          2
13523      0          2

[3621 rows x 2 columns]
{0: 'C', 1: 'CT', 2: 'O', 3: 'E'}
              precision    recall  f1-score   support

           C       0.23      0.82      0.36       465
          CT       0.00      0.00      0.00        25
           O       0.82      0.20      0.32      2663
           E       0.21      0.55      0.30       468

    accuracy                           0.32      3621
   macro avg       0.32      0.39      0.25      3621
weighted avg       0.66      0.32      0.32      3621



Epoch,Training Loss,Validation Loss
1,0.000200,1.303558
2,0.000100,1.329777
3,0.000100,1.323389


       preds  gold_data
1          0          0
2          0          0
5          0          0
6          0          0
7          0          0
...      ...        ...
13519      3          0
13520      0          0
13521      0          2
13522      0          2
13523      0          2

[3621 rows x 2 columns]
{0: 'C', 1: 'CT', 2: 'O', 3: 'E'}
              precision    recall  f1-score   support

           C       0.24      0.82      0.37       465
          CT       0.00      0.00      0.00        25
           O       0.83      0.25      0.38      2663
           E       0.22      0.52      0.31       468

    accuracy                           0.35      3621
   macro avg       0.32      0.40      0.27      3621
weighted avg       0.67      0.35      0.37      3621



Epoch,Training Loss,Validation Loss
1,0.000100,1.325031
2,0.000700,1.352258
3,0.002100,1.401195


       preds  gold_data
1          0          0
2          0          0
5          0          0
6          0          0
7          0          0
...      ...        ...
13519      3          0
13520      0          0
13521      0          2
13522      0          2
13523      0          2

[3621 rows x 2 columns]
{0: 'C', 1: 'CT', 2: 'O', 3: 'E'}
              precision    recall  f1-score   support

           C       0.24      0.81      0.37       465
          CT       0.00      0.00      0.00        25
           O       0.83      0.22      0.35      2663
           E       0.21      0.56      0.31       468

    accuracy                           0.34      3621
   macro avg       0.32      0.40      0.26      3621
weighted avg       0.67      0.34      0.34      3621



In [ ]:
############Organizational data as train and FinCausal as test ###########################

In [ ]:
data_test = pd.read_json('/content/Fincausal-test.json', lines=True)
data_test.head(4)

,words,ner,id,postfix
0,"[During, 2020, ,, the, Company, also, issued, ...","[E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, ...",3,"[ , , , , , , , , , , , , , , , , ..."
1,"[The, significant, increases, in, earnings, as...","[E, E, E, E, E, E, E, E, O, O, O, O, C, C, C, ...",4,"[ , , , , , , , , , , , , , , , ..."
2,"[The, provision, for, credit, losses, was, $, ...","[E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, ...",7,"[ , , , , , , , , , , , , , , , , ..."
3,"[Higher, net, charge-offs, also, contributed, ...","[E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, ...",8,"[ , , , , , , , , , , , , , , , ..."


In [ ]:
# load data and rename relevant columns to X (input) and y (output)
dataset_test = data_test[["words", "ner"]].rename(columns={"words": "A", "ner": "B"})
# load each row's data as a list of strings
s1, s2 = [json.dumps(tokens_test) for tokens_test in dataset_test.A.values], [json.dumps(labels_test) for labels_test in dataset_test.B.values]

X_raw_test, y_raw_test = [json.loads(token_test) for token_test in s1], [json.loads(label_test) for label_test in s2]


In [ ]:
X_test= tokenize(X_raw_test)

In [ ]:
y_test = encode_tags(tags=y_raw_test, encodings=X_test, tag2id=tag2id)

In [ ]:
test_dataset = CustomDataset(X_test,y_test)

In [ ]:
for i in range(10):
  trainer = Trainer(
    model=model,                          # the instantiated 🤗 Transformers model to be trained
    args=training_args,                   # training arguments, defined above
    train_dataset=train_dataset,          # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    #compute_metrics = compute_metrics
  )

  trainer.train()
  pred = trainer.predict(test_dataset)
  preds = pred.predictions.argmax(-1)
  flat_list = [item for sublist in preds for item in sublist]
  flat_list1 = [item1 for sublist1 in y_test for item1 in sublist1]

  lis_preds_frame = pd.DataFrame(flat_list)
  lis_test_labels_frame = pd.DataFrame(flat_list1)
  frame_combine_test = pd.concat([lis_preds_frame,lis_test_labels_frame],axis=1)
  frame_combine_test.columns = ['preds','gold_data']

  frame_combine_test_new = frame_combine_test[frame_combine_test.gold_data != -100]
  print(frame_combine_test_new)

  print(id2tag)
  targetname = ['C','CT', 'O','E']


  print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()

Epoch,Training Loss,Validation Loss
1,0.003600,1.377100
2,0.000100,1.385974
3,0.002200,1.368217


       preds  gold_data
1          2          3
2          2          3
3          2          3
4          0          3
5          0          3
...      ...        ...
55505      0          0
55506      0          0
55507      0          0
55508      0          0
55509      0          0

[12309 rows x 2 columns]
{0: 'C', 1: 'CT', 2: 'O', 3: 'E'}
              precision    recall  f1-score   support

           C       0.76      0.71      0.74      6590
          CT       0.00      1.00      0.00         0
           O       0.13      0.29      0.18       640
           E       0.68      0.57      0.62      5079

    accuracy                           0.63     12309
   macro avg       0.39      0.64      0.38     12309
weighted avg       0.70      0.63      0.66     12309



Epoch,Training Loss,Validation Loss
1,0.011700,1.379128
2,0.000000,1.386622
3,0.004500,1.347990


       preds  gold_data
1          2          3
2          2          3
3          2          3
4          0          3
5          0          3
...      ...        ...
55505      0          0
55506      0          0
55507      0          0
55508      0          0
55509      0          0

[12309 rows x 2 columns]
{0: 'C', 1: 'CT', 2: 'O', 3: 'E'}
              precision    recall  f1-score   support

           C       0.78      0.67      0.72      6590
          CT       0.00      1.00      0.00         0
           O       0.12      0.33      0.18       640
           E       0.67      0.59      0.63      5079

    accuracy                           0.62     12309
   macro avg       0.39      0.65      0.38     12309
weighted avg       0.70      0.62      0.65     12309



Epoch,Training Loss,Validation Loss
1,0.002600,1.329511
2,0.000100,1.390411
3,0.000400,1.390771


...

       preds  gold_data
1          2          3
2          2          3
3          2          3
4          0          3
5          0          3
...      ...        ...
55505      0          0
55506      0          0
55507      0          0
55508      0          0
55509      3          0

[12309 rows x 2 columns]
{0: 'C', 1: 'CT', 2: 'O', 3: 'E'}
              precision    recall  f1-score   support

           C       0.85      0.64      0.73      6590
          CT       0.00      1.00      0.00         0
           O       0.15      0.27      0.19       640
           E       0.65      0.74      0.70      5079

    accuracy                           0.66     12309
   macro avg       0.41      0.66      0.40     12309
weighted avg       0.73      0.66      0.69     12309



Epoch,Training Loss,Validation Loss
1,0.004600,1.453717
2,0.001000,1.449744
3,0.004800,1.505945


       preds  gold_data
1          2          3
2          2          3
3          2          3
4          0          3
5          0          3
...      ...        ...
55505      0          0
55506      0          0
55507      0          0
55508      0          0
55509      3          0

[12309 rows x 2 columns]
{0: 'C', 1: 'CT', 2: 'O', 3: 'E'}
              precision    recall  f1-score   support

           C       0.83      0.69      0.75      6590
          CT       0.00      1.00      0.00         0
           O       0.13      0.29      0.18       640
           E       0.70      0.68      0.69      5079

    accuracy                           0.66     12309
   macro avg       0.41      0.66      0.41     12309
weighted avg       0.74      0.66      0.70     12309



Epoch,Training Loss,Validation Loss
1,0.000400,1.460862
2,0.000100,1.513491
3,0.001400,1.509386


       preds  gold_data
1          2          3
2          2          3
3          2          3
4          0          3
5          0          3
...      ...        ...
55505      0          0
55506      0          0
55507      0          0
55508      0          0
55509      3          0

[12309 rows x 2 columns]
{0: 'C', 1: 'CT', 2: 'O', 3: 'E'}
              precision    recall  f1-score   support

           C       0.83      0.66      0.74      6590
          CT       0.00      1.00      0.00         0
           O       0.13      0.29      0.18       640
           E       0.67      0.68      0.67      5079

    accuracy                           0.65     12309
   macro avg       0.41      0.66      0.40     12309
weighted avg       0.72      0.65      0.68     12309



In [ ]:
!python run_ner.py --model_name_or_path SpanBERT/spanbert-large-cased   \
           --train_file /content/content/data/fin_causal_2022/fincausal-extra-all-train.json  \
           --validation_file /content/content/data/fin_causal_2022/new_data/task2_2022_addition-dev.json  \
           --test_file /content/content/data/fin_causal_2022/new_data/task2_2022_addition-dev.json  \
           --output_dir /content/content/data/test-ner-2022 --do_train --do_eval --do_predict --learning_rate 5e-5 --num_train_epochs 30 --weight_decay 0 --per_device_train_batch_size 16  #--classifier_dropout 0.1